In [1]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=ca8a459b41cdae2190a5880a46b2be9ba6fd48363c0e78ac252be1024d068d18
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=89aa98a52cbe11ad0e84187221a06a1aae405301147f98169d63d29dabeb0370
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [3]:
pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 4.6 MB/s eta 0:00:00


In [4]:
pip install SMOTE

In [5]:
pip install imbalanced-learn

In [6]:
pip install joblib

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import ta  # Technical Analysis library
import pandas_ta as pata
import shap
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
import joblib

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


######################

**테스트 데이터 생성**
######################

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_data.csv'
data_test = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

,open_time,open,high,low,close,volume
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91


In [ ]:
# 데이터프레임의 길이 계산
total_length = len(data_test)

# 마지막 1/3 데이터 분리
data_test = data_test.iloc[-(total_length // 3):]

In [ ]:
data_test

,open_time,open,high,low,close,volume
1237691,2023-05-10 20:56:00,21.27,21.27,21.24,21.24,724.800
1237692,2023-05-10 20:57:00,21.25,21.26,21.25,21.25,1058.840
1237693,2023-05-10 20:58:00,21.25,21.25,21.23,21.24,2098.960
1237694,2023-05-10 20:59:00,21.24,21.25,21.20,21.22,7424.210
1237695,2023-05-10 21:00:00,21.21,21.24,21.16,21.18,5169.380
...,...,...,...,...,...,...
1856531,2024-07-13 14:56:00,139.93,140.05,139.93,139.99,929.251
1856532,2024-07-13 14:57:00,139.98,139.99,139.90,139.90,71.095
1856533,2024-07-13 14:58:00,139.91,139.93,139.88,139.90,171.879
1856534,2024-07-13 14:59:00,139.90,139.95,139.89,139.91,425.029


######################

**테스트 데이터 로딩**
######################

In [ ]:
data_test = pd.read_csv("/content/drive/MyDrive/Data/SOL_Data_Test.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

,open_time,open,high,low,close,volume
0,2024-07-14 15:00:00,145.02,145.20,145.00,145.20,1788.764
1,2024-07-14 15:01:00,145.21,145.50,145.18,145.49,2510.833
2,2024-07-14 15:02:00,145.50,145.57,145.45,145.45,1708.171
3,2024-07-14 15:03:00,145.45,145.49,145.32,145.44,1655.282
4,2024-07-14 15:04:00,145.44,145.63,145.44,145.60,2003.321


######################

**테스트 데이터 처리**
######################

In [ ]:
# open_time 열을 datetime 형식으로 변환
data_test['open_time'] = pd.to_datetime(data_test['open_time'])

# 시각과 분만 추출하여 time 열 생성
data_test['time'] = data_test['open_time'].dt.strftime('%H:%M')

# ATR 계산
atr_periods = [5, 10, 14, 20, 50]
for period in atr_periods:
    data_test[f'atr_{period}'] = ta.volatility.average_true_range(data_test['high'], data_test['low'], data_test['close'], window=period)

# VWAP 계산
data_test['vwap'] = ta.volume.volume_weighted_average_price(data_test['high'], data_test['low'], data_test['close'], data_test['volume'])

# Stochastic Oscillator 계산
stoch_periods = [(14, 3), (21, 5), (9, 3), (5, 2), (20, 7)]
for period, smooth in stoch_periods:
    data_test[f'stoch_%k_{period}_{smooth}'] = ta.momentum.stoch(data_test['high'], data_test['low'], data_test['close'], window=period, smooth_window=smooth)
    data_test[f'stoch_%d_{period}_{smooth}'] = ta.momentum.stoch_signal(data_test['high'], data['low'], data_test['close'], window=period, smooth_window=smooth)

# OBV 계산
data_test['obv'] = ta.volume.on_balance_volume(data_test['close'], data_test['volume'])


# Bollinger Bands 계산
bollinger_periods = [10, 20, 50, 100, 200]
for period in bollinger_periods:
    data_test[f'bollinger_hband_{period}'] = ta.volatility.BollingerBands(data_test['close'], window=period).bollinger_hband()
    data_test[f'bollinger_lband_{period}'] = ta.volatility.BollingerBands(data_test['close'], window=period).bollinger_lband()

# Ichimoku 계산
ichimoku_periods = [9, 26, 52, 100, 200]
for period in ichimoku_periods:
    data_test[f'ichimoku_base_{period}'] = ta.trend.ichimoku_base_line(data_test['high'], data_test['low'], window1=period)
    data_test[f'ichimoku_conversion_{period}'] = ta.trend.ichimoku_conversion_line(data_test['high'], data_test['low'], window1=period)
# Supertrend 계산 함수
def calculate_supertrend(df, period=7, multiplier=3, atr_period=14):
    df = df.copy()  # 데이터프레임을 복사하여 사용
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=atr_period)
    df['upperband'] = hl2 + (multiplier * df['atr'])
    df['lowerband'] = hl2 - (multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in tqdm(range(1, len(df.index)), desc=f'Calculating Supertrend {period}-{multiplier}-{atr_period}'):
        previous = current - 1

        if df['close'].iloc[current] > df['upperband'].iloc[previous]:
            df.loc[df.index[current], 'in_uptrend'] = True
        elif df['close'].iloc[current] < df['lowerband'].iloc[previous]:
            df.loc[df.index[current], 'in_uptrend'] = False
        else:
            df.loc[df.index[current], 'in_uptrend'] = df['in_uptrend'].iloc[previous]

            if df['in_uptrend'].iloc[current] and df['lowerband'].iloc[current] < df['lowerband'].iloc[previous]:
                df.loc[df.index[current], 'lowerband'] = df['lowerband'].iloc[previous]

            if not df['in_uptrend'].iloc[current] and df['upperband'].iloc[current] > df['upperband'].iloc[previous]:
                df.loc[df.index[current], 'upperband'] = df['upperband'].iloc[previous]

    return df

# Supertrend 계산
supertrend_settings = [(7, 3, 14), (10, 3, 20), (14, 2, 10), (20, 4, 50), (50, 5, 5)]
for period, multiplier, atr_period in supertrend_settings:
    data_test = calculate_supertrend(data_test, period, multiplier, atr_period)
    data_test[f'supertrend_upper_{period}_{multiplier}_{atr_period}'] = data_test['upperband']
    data_test[f'supertrend_lower_{period}_{multiplier}_{atr_period}'] = data_test['lowerband']
    data_test[f'supertrend_in_uptrend_{period}_{multiplier}_{atr_period}'] = data_test['in_uptrend']

Calculating Supertrend 50-5-5: 100%|██████████| 8640/8640 [00:03<00:00, 2877.82it/s]


######################

**테스트 데이터 60분 처리**
######################

In [ ]:
def calculate_max_min_returns(df):
    window_size = 60

    # 'open_time' 열이 데이터프레임에 있는지 확인
    if 'open_time' not in df.columns:
        raise KeyError("'open_time' 열이 데이터프레임에 포함되어 있어야 합니다.")

    # 'open_time' 열을 datetime으로 변환
    df['open_time'] = pd.to_datetime(df['open_time'])

    # 인덱스 중복 확인 및 제거
    df = df[~df.duplicated(subset='open_time', keep='first')].copy()

    # 각 행에 대해 60분 윈도우를 적용하여 최대 및 최소 가격 계산
    df['max_price'] = df.apply(lambda row: df[(df['open_time'] >= row['open_time']) &
                                              (df['open_time'] < row['open_time'] + pd.Timedelta(minutes=window_size))]['high'].max(), axis=1)
    df['min_price'] = df.apply(lambda row: df[(df['open_time'] >= row['open_time']) &
                                              (df['open_time'] < row['open_time'] + pd.Timedelta(minutes=window_size))]['low'].min(), axis=1)

    # 결측값을 적절히 처리
    df['max_price'].fillna(df['high'], inplace=True)
    df['min_price'].fillna(df['low'], inplace=True)

    # 현재 가격
    current_price = df['close']

    # 최대 및 최소 수익률 계산
    df['max_return_60min'] = ((df['max_price'] - current_price) / current_price) * 100
    df['min_return_60min'] = ((df['min_price'] - current_price) / current_price) * 100

    # 필요없는 열 삭제
    df.drop(columns=['max_price', 'min_price'], inplace=True)

    return df

# 최대 상승률과 최대 하락률 계산
data_test = calculate_max_min_returns(data_test)

######################

**테스트 추가 데이터 처리**
######################

In [ ]:
# 지표 계산 함수 정의

def calculate_indicators(df):
    # Parabolic SAR
    df['Parabolic_SAR_0.02'] = pata.psar(df['high'], df['low'], df['close'], af=0.02, max_af=0.2)['PSARl_0.02_0.2']
    df['Parabolic_SAR_0.04'] = pata.psar(df['high'], df['low'], df['close'], af=0.04, max_af=0.2)['PSARl_0.04_0.2']
    df['Parabolic_SAR_0.06'] = pata.psar(df['high'], df['low'], df['close'], af=0.06, max_af=0.2)['PSARl_0.06_0.2']
    df['Parabolic_SAR_0.08'] = pata.psar(df['high'], df['low'], df['close'], af=0.08, max_af=0.2)['PSARl_0.08_0.2']
    df['Parabolic_SAR_0.1'] = pata.psar(df['high'], df['low'], df['close'], af=0.1, max_af=0.2)['PSARl_0.1_0.2']

    # Williams %R
    df['Williams_%R_10'] = pata.willr(df['high'], df['low'], df['close'], length=10)
    df['Williams_%R_20'] = pata.willr(df['high'], df['low'], df['close'], length=20)
    df['Williams_%R_30'] = pata.willr(df['high'], df['low'], df['close'], length=30)
    df['Williams_%R_40'] = pata.willr(df['high'], df['low'], df['close'], length=40)
    df['Williams_%R_50'] = pata.willr(df['high'], df['low'], df['close'], length=50)

    # Momentum
    df['Momentum_10'] = pata.mom(df['close'], length=10)
    df['Momentum_20'] = pata.mom(df['close'], length=20)
    df['Momentum_30'] = pata.mom(df['close'], length=30)
    df['Momentum_40'] = pata.mom(df['close'], length=40)
    df['Momentum_50'] = pata.mom(df['close'], length=50)

    # Rate of Change (ROC)
    df['ROC_10'] = pata.roc(df['close'], length=10)
    df['ROC_20'] = pata.roc(df['close'], length=20)
    df['ROC_30'] = pata.roc(df['close'], length=30)
    df['ROC_40'] = pata.roc(df['close'], length=40)
    df['ROC_50'] = pata.roc(df['close'], length=50)

    # Chande Momentum Oscillator (CMO)
    df['CMO_10'] = pata.cmo(df['close'], length=10)
    df['CMO_20'] = pata.cmo(df['close'], length=20)
    df['CMO_30'] = pata.cmo(df['close'], length=30)
    df['CMO_40'] = pata.cmo(df['close'], length=40)
    df['CMO_50'] = pata.cmo(df['close'], length=50)

    # Money Flow Index (MFI)
    df['MFI_10'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=10)
    df['MFI_20'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=20)
    df['MFI_30'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=30)
    df['MFI_40'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=40)
    df['MFI_50'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=50)

    # Relative Strength Index (RSI)
    df['RSI_10'] = pata.rsi(df['close'], length=10)
    df['RSI_20'] = pata.rsi(df['close'], length=20)
    df['RSI_30'] = pata.rsi(df['close'], length=30)
    df['RSI_40'] = pata.rsi(df['close'], length=40)
    df['RSI_50'] = pata.rsi(df['close'], length=50)

    # Accumulation/Distribution Line (A/D Line)
    df['Accumulation_Distribution_Line'] = pata.ad(df['high'], df['low'], df['close'], df['volume'])

    # Elder's Force Index (EFI)
    df['Elder_Force_Index_2'] = pata.efi(df['close'], df['volume'], length=2)
    df['Elder_Force_Index_13'] = pata.efi(df['close'], df['volume'], length=13)
    df['Elder_Force_Index_5'] = pata.efi(df['close'], df['volume'], length=5)
    df['Elder_Force_Index_10'] = pata.efi(df['close'], df['volume'], length=10)
    df['Elder_Force_Index_25'] = pata.efi(df['close'], df['volume'], length=25)

    # Relative Vigor Index (RVI)
    df['Relative_Vigor_Index_10'] = pata.rsi(df['close'], length=10)  # pandas-ta는 RVI가 없으므로 RSI로 대체
    df['Relative_Vigor_Index_20'] = pata.rsi(df['close'], length=20)
    df['Relative_Vigor_Index_30'] = pata.rsi(df['close'], length=30)
    df['Relative_Vigor_Index_40'] = pata.rsi(df['close'], length=40)
    df['Relative_Vigor_Index_50'] = pata.rsi(df['close'], length=50)

    # Volume Ratio (VR)
    def volume_ratio(close, volume, period):
        vr = []
        for i in range(len(close)):
            if i < period:
                vr.append(None)
            else:
                vol_up = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] > close[j - 1])
                vol_down = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] < close[j - 1])
                vol_same = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] == close[j - 1])
                vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
                vr.append(vr_value)
        return vr

    df['VR_10'] = volume_ratio(df['close'], df['volume'], period=10)
    df['VR_20'] = volume_ratio(df['close'], df['volume'], period=20)
    df['VR_30'] = volume_ratio(df['close'], df['volume'], period=30)
    df['VR_40'] = volume_ratio(df['close'], df['volume'], period=40)
    df['VR_50'] = volume_ratio(df['close'], df['volume'], period=50)

    # Commodity Channel Index (CCI)
    df['CCI_10'] = pata.cci(df['high'], df['low'], df['close'], length=10)
    df['CCI_20'] = pata.cci(df['high'], df['low'], df['close'], length=20)
    df['CCI_30'] = pata.cci(df['high'], df['low'], df['close'], length=30)
    df['CCI_40'] = pata.cci(df['high'], df['low'], df['close'], length=40)
    df['CCI_50'] = pata.cci(df['high'], df['low'], df['close'], length=50)

    return df

# 지표 계산
data_test = calculate_indicators(data_test)

<ipython-input-35-704f26e2c291>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100


In [ ]:
# 이격도(Price Disparity Index) 계산 함수
def calculate_disparity_index(df, period):
    df[f'disparity_index_{period}'] = (df['close'] / df['close'].rolling(window=period).mean()) * 100
    return df

# 이평선(이동평균선) 계산 함수
def calculate_moving_averages(df, periods):
    for period in periods:
        df[f'price_ma_{period}'] = df['close'].rolling(window=period).mean()
        df[f'volume_ma_{period}'] = df['volume'].rolling(window=period).mean()
    return df

# 이격도와 이동평균선 계산
disparity_periods = [5, 10, 20, 50, 100, 200]
moving_average_periods = [5, 10, 20, 50, 100, 200]

# 이격도 계산
for period in disparity_periods:
    data_test = calculate_disparity_index(data_test, period)

# 이동평균선 계산
data_test = calculate_moving_averages(data_test, moving_average_periods)

######################

**테스트 처리 데이터 저장**
######################

In [ ]:
# Load the dataset from Kaggle
path2 = '/content/drive/MyDrive/Data/SOL_Data_Test_Indicator3.csv'
data_test.to_csv(path2, index=True);

######################

**초기 데이터 처리**
######################

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,open_time,open,high,low,close,volume
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91


In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

######################

**1/3 데이터 처리**
######################

In [ ]:
# 데이터의 첫 번째 1/3만 사용
n_samples = len(data)
data = data.iloc[:n_samples // 3]

######################

**가용 지표 입력**
######################

In [ ]:
# open_time 열을 datetime 형식으로 변환
data['open_time'] = pd.to_datetime(data['open_time'])

# 시각과 분만 추출하여 time 열 생성
data['time'] = data['open_time'].dt.strftime('%H:%M')

# ATR 계산
atr_periods = [5, 10, 14, 20, 50]
for period in atr_periods:
    data[f'atr_{period}'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'], window=period)

# VWAP 계산
data['vwap'] = ta.volume.volume_weighted_average_price(data['high'], data['low'], data['close'], data['volume'])

# Stochastic Oscillator 계산
stoch_periods = [(14, 3), (21, 5), (9, 3), (5, 2), (20, 7)]
for period, smooth in stoch_periods:
    data[f'stoch_%k_{period}_{smooth}'] = ta.momentum.stoch(data['high'], data['low'], data['close'], window=period, smooth_window=smooth)
    data[f'stoch_%d_{period}_{smooth}'] = ta.momentum.stoch_signal(data['high'], data['low'], data['close'], window=period, smooth_window=smooth)

# Supertrend 계산 함수
def calculate_supertrend(df, period=7, multiplier=3, atr_period=14):
    df = df.copy()  # 데이터프레임을 복사하여 사용
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=atr_period)
    df['upperband'] = hl2 + (multiplier * df['atr'])
    df['lowerband'] = hl2 - (multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in tqdm(range(1, len(df.index)), desc=f'Calculating Supertrend {period}-{multiplier}-{atr_period}'):
        previous = current - 1

        if df['close'].iloc[current] > df['upperband'].iloc[previous]:
            df.loc[df.index[current], 'in_uptrend'] = True
        elif df['close'].iloc[current] < df['lowerband'].iloc[previous]:
            df.loc[df.index[current], 'in_uptrend'] = False
        else:
            df.loc[df.index[current], 'in_uptrend'] = df['in_uptrend'].iloc[previous]

            if df['in_uptrend'].iloc[current] and df['lowerband'].iloc[current] < df['lowerband'].iloc[previous]:
                df.loc[df.index[current], 'lowerband'] = df['lowerband'].iloc[previous]

            if not df['in_uptrend'].iloc[current] and df['upperband'].iloc[current] > df['upperband'].iloc[previous]:
                df.loc[df.index[current], 'upperband'] = df['upperband'].iloc[previous]

    return df

# Supertrend 계산
supertrend_settings = [(7, 3, 14), (10, 3, 20), (14, 2, 10), (20, 4, 50), (50, 5, 5)]
for period, multiplier, atr_period in supertrend_settings:
    data = calculate_supertrend(data, period, multiplier, atr_period)
    data[f'supertrend_upper_{period}_{multiplier}_{atr_period}'] = data['upperband']
    data[f'supertrend_lower_{period}_{multiplier}_{atr_period}'] = data['lowerband']
    data[f'supertrend_in_uptrend_{period}_{multiplier}_{atr_period}'] = data['in_uptrend']

######################

**비가용 지표**
######################

In [ ]:
# Bollinger Bands 계산
bollinger_periods = [10, 20, 50, 100, 200]
for period in bollinger_periods:
    data[f'bollinger_hband_{period}'] = ta.volatility.BollingerBands(data['close'], window=period).bollinger_hband()
    data[f'bollinger_lband_{period}'] = ta.volatility.BollingerBands(data['close'], window=period).bollinger_lband()

# OBV 계산
data['obv'] = ta.volume.on_balance_volume(data['close'], data['volume'])

# Ichimoku 계산
ichimoku_periods = [9, 26, 52, 100, 200]
for period in ichimoku_periods:
    data[f'ichimoku_base_{period}'] = ta.trend.ichimoku_base_line(data['high'], data['low'], window1=period)
    data[f'ichimoku_conversion_{period}'] = ta.trend.ichimoku_conversion_line(data['high'], data['low'], window1=period)


######################

**60봉 이내 최대 상승 및 하락 %**
######################

In [ ]:
def calculate_max_min_returns(df):
    window_size = 60

    # 'open_time'이 이미 인덱스로 설정되어 있는지 확인
    if df.index.name != 'open_time':
        raise KeyError("'open_time' 열이 인덱스로 설정되어 있어야 합니다.")

    # 인덱스 중복 확인 및 제거
    df = df[~df.index.duplicated(keep='first')].copy()

    # 60분 윈도우를 적용하여 최대 및 최소 가격 계산
    df['max_price'] = df['high'].rolling(f'{window_size}T').max().shift(-window_size)
    df['min_price'] = df['low'].rolling(f'{window_size}T').min().shift(-window_size)

    # 결측값을 적절히 처리 (예: 마지막 몇 행)
    df['max_price'].fillna(df['high'], inplace=True)
    df['min_price'].fillna(df['low'], inplace=True)

    # 현재 가격
    current_price = df['close']

    # 최대 및 최소 수익률 계산
    df['max_return_60min'] = ((df['max_price'] - current_price) / current_price) * 100
    df['min_return_60min'] = ((df['min_price'] - current_price) / current_price) * 100

    # 필요없는 열 삭제
    df.drop(columns=['max_price', 'min_price'], inplace=True)

    return df

# 최대 상승률과 최대 하락률 계산
data = calculate_max_min_returns(data)

In [ ]:
# 인덱스를 리셋하여 'open_time'을 열로 되돌림
data.reset_index(inplace=True)

data.head()

In [ ]:
data.columns

######################

**처리 데이터 저장**
######################

In [ ]:
path2 = "/content/drive/MyDrive/Data/SOL60_INDICATOR2_SMALL";
data.to_csv(path2, index=True);

######################

**테스트 처리 데이터 로딩(S)**
######################

In [2]:
data_test = pd.read_csv("/content/drive/MyDrive/Data/SOL_Data_Test_Indicator3.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,price_ma_10,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200
0,0,2024-07-14 15:00:00,145.02,145.20,145.00,145.20,1788.764,15:00,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-07-14 15:01:00,145.21,145.50,145.18,145.49,2510.833,15:01,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2024-07-14 15:02:00,145.50,145.57,145.45,145.45,1708.171,15:02,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2024-07-14 15:03:00,145.45,145.49,145.32,145.44,1655.282,15:03,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2024-07-14 15:04:00,145.44,145.63,145.44,145.60,2003.321,15:04,0.2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#2nd Account
data_test = pd.read_csv("/content/drive/MyDrive/SOL_Data_Test_Indicator3.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

In [ ]:
data_test = pd.read_csv("/content/drive/MyDrive/Data/SOL_Data_Test_Indicator.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_lower_14_2_10,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min
0,0,2024-07-14 15:00:00,145.02,145.20,145.00,145.20,1788.764,15:00,0.0,0.0,...,145.100,True,145.100,145.100,True,145.100,145.100,True,0.578512,-0.399449
1,1,2024-07-14 15:01:00,145.21,145.50,145.18,145.49,2510.833,15:01,0.0,0.0,...,145.340,True,145.340,145.340,True,145.340,145.340,True,0.378033,-0.597979
2,2,2024-07-14 15:02:00,145.50,145.57,145.45,145.45,1708.171,15:02,0.0,0.0,...,145.510,True,145.510,145.510,True,145.510,145.510,True,0.405638,-0.570643
3,3,2024-07-14 15:03:00,145.45,145.49,145.32,145.44,1655.282,15:03,0.0,0.0,...,145.405,False,145.405,145.405,False,145.405,145.405,False,0.412541,-0.563806
4,4,2024-07-14 15:04:00,145.44,145.63,145.44,145.60,2003.321,15:04,0.2,0.0,...,145.535,True,145.535,145.535,True,146.535,144.535,True,0.302198,-0.673077


In [ ]:
#2nd Account
data_test = pd.read_csv("/content/drive/MyDrive/SOL_Data_Test_Indicator.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SOL_Data_Test_Indicator.csv'

######################

**테스트 처리 데이터 로딩(L)**
######################

In [2]:
data_test = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR3")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

,Unnamed: 0.1,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,...,price_ma_10,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200
0,0,0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#2nd Account
data_test = pd.read_csv("/content/drive/MyDrive/SOL60_INDICATOR3")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SOL60_INDICATOR3'

In [ ]:
data_test = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR2.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_lower_14_2_10,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min
0,0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,0.0,...,1.44510,True,1.44510,1.44510,True,1.44510,1.44510,True,1.203652,-0.387382
1,1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,0.0,...,1.44705,True,1.44705,1.44705,True,1.44705,1.44705,True,1.154671,-0.435594
2,2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,0.0,...,1.44980,True,1.44980,1.44980,True,1.44980,1.44980,True,1.008009,-0.579950
3,3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,0.0,...,1.45310,True,1.45310,1.45310,True,1.45310,1.45310,True,0.494573,-1.085314
4,4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,0.0,...,1.45570,True,1.45570,1.45570,True,1.47590,1.43550,True,0.418697,-1.159997


In [ ]:
#2nd Account
data_test = pd.read_csv("/content/drive/MyDrive/SOL60_INDICATOR2_SMALL")

# Display the first few rows of the dataset to ensure it is loaded correctly
data_test.head()

NameError: name 'pd' is not defined

In [3]:
#3등분 한 것의 마지막 등분
# 데이터프레임의 길이 계산
total_length = len(data_test)

# 마지막 1/3 데이터 분리
data_test = data_test.iloc[-(total_length // 3):]

In [3]:
#3등분 한 것의 두번째
# 데이터프레임의 길이 계산
total_length = len(data_test)

# 3등분으로 나눈 길이 계산
split_length = total_length // 3

# 중간 부분 데이터 분리
data_test = data_test.iloc[split_length:split_length*2]

######################

**테스트 처리 전처리(공통)**
######################

In [4]:
# open_time 열을 datetime 형식으로 변환
if not np.issubdtype(data_test['open_time'].dtype, np.datetime64):
    data_test['open_time'] = pd.to_datetime(data_test['open_time'])

# time 열을 분 단위로 변환
data_test['time'] = data_test['open_time'].dt.hour * 60 + data_test['open_time'].dt.minute

In [5]:
data_test.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'open_time', 'open', 'high', 'low',
       'close', 'volume', 'time', 'atr_5',
       ...
       'price_ma_10', 'volume_ma_10', 'price_ma_20', 'volume_ma_20',
       'price_ma_50', 'volume_ma_50', 'price_ma_100', 'volume_ma_100',
       'price_ma_200', 'volume_ma_200'],
      dtype='object', length=141)

In [5]:
data_test = data_test.drop(columns=['Unnamed: 0'])

In [6]:
data_test = data_test.drop(columns=['Unnamed: 0.1'])

######################

**테스트 처리 전처리(시계열)**
######################

In [ ]:
# 사용하지 않을 열 제외
data_test_predict = data_test.drop(columns=['open_time', 'max_return_60min', 'min_return_60min'])

In [ ]:
# 무한대 값을 NaN으로 대체
data_test_predict.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균으로 대체
imputer = SimpleImputer(strategy='mean')
data_test_predict_imputed = imputer.fit_transform(data_test_predict)  # 같은 imputer 사용

# 데이터 정규화
scaler = MinMaxScaler()
data_test_predict_scaled = scaler.fit_transform(data_test_predict_imputed)  # 같은 scaler 사용

# 예측 데이터를 시퀀스 형태로 변환 (LSTM용)
def create_sequences_for_prediction(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

# 시퀀스 길이 설정
sequence_length = 60

# 예측용 시퀀스 데이터 생성
X_test_seq = create_sequences_for_prediction(data_test_predict_scaled, sequence_length)


######################

**테스트 처리 전처리(비시계열)**
######################

In [13]:
data_test.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'time', 'atr_5',
       'atr_10', 'atr_14',
       ...
       'price_ma_10', 'volume_ma_10', 'price_ma_20', 'volume_ma_20',
       'price_ma_50', 'volume_ma_50', 'price_ma_100', 'volume_ma_100',
       'price_ma_200', 'volume_ma_200'],
      dtype='object', length=139)

In [ ]:
data_test = data_test.drop(columns=['Unnamed: 0'])

In [ ]:
data_test = data_test.drop(columns=['Unnamed: 0.1'])

In [ ]:
# target 컬럼과 max_return_60min, min_return_60min 컬럼 제거 (목표 변수를 제외하고 피처 선택을 위해)
selected_columns = [
    "obv", "volume_ma_200", "Accumulation_Distribution_Line", "volume_ma_100", "volume_ma_50", "time",
    "atr_50", "atr_20", "volume_ma_20", "disparity_index_200", "bollinger_lband_200", "volume_ma_10",
    "bollinger_hband_200", "atr_14", "price_ma_200", "ichimoku_conversion_200", "bollinger_lband_100",
    "atr_10", "bollinger_hband_100", "disparity_index_100", "price_ma_100", "ichimoku_conversion_100",
    "volume_ma_5", "bollinger_lband_50", "bollinger_hband_50", "VR_50", "price_ma_50", "ichimoku_conversion_52",
    "atr_5", "MFI_50", "ROC_50", "supertrend_lower_20_4_50", "atr", "VR_40", "supertrend_lower_50_5_5", "MFI_40",
    "Elder_Force_Index_25", "lowerband", "ROC_40", "bollinger_lband_20", "supertrend_upper_20_4_50", "VR_30",
    "supertrend_lower_7_3_14"
]

# 선택된 컬럼만으로 데이터프레임 생성
data_test_predict = data_test[selected_columns]

# 무한대 값을 NaN으로 변환
data_test_predict.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값 처리
imputer = SimpleImputer(strategy='mean')
data_test_predict_imputed = imputer.fit_transform(data_test_predict)

# 데이터 정규화
scaler = StandardScaler()
data_test_predict_scaled = scaler.fit_transform(data_test_predict_imputed)

<ipython-input-4-5597080ee8ac>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_predict.replace([np.inf, -np.inf], np.nan, inplace=True)


In [7]:
# 무한대 값을 NaN으로 대체
data_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# 사용하지 않을 열 제외
data_test_predict = data_test.drop(columns=['open_time', 'max_return_60min', 'min_return_60min'])

# NaN 값을 평균으로 대체
imputer = SimpleImputer(strategy='mean')
data_test_predict_imputed = imputer.fit_transform(data_test_predict)

# 데이터 정규화
scaler = StandardScaler()
data_test_predict_scaled = scaler.fit_transform(data_test_predict_imputed)

######################

**처리 데이터 로딩**
######################

In [ ]:
#메인 데이터
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR3")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,Unnamed: 0.1,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,...,price_ma_10,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200
0,0,0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#2nd 계정 로드
#메인 데이터
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR3")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
#3등분 한 것의 두번째
# 데이터프레임의 길이 계산
total_length = len(data)

# 3등분으로 나눈 길이 계산
split_length = total_length // 3

# 중간 부분 데이터 분리
data = data.iloc[split_length:split_length*2]
data.head()

In [ ]:
#3등분 한 것의 마지막 등분
# 데이터프레임의 길이 계산
total_length = len(data)

# 마지막 1/3 데이터 분리
data = data.iloc[-(total_length // 3):]
data.head()

In [22]:
#메인 데이터_SMALL
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR3_SMALL")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,Unnamed: 0.1,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,...,price_ma_10,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200
0,0,0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#2nd 계정 로드
#메인 데이터_SMALL
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR3_SMALL")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
#메인 데이터
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR2.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_lower_14_2_10,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min
0,0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,0.0,...,1.44510,True,1.44510,1.44510,True,1.44510,1.44510,True,1.203652,-0.387382
1,1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,0.0,...,1.44705,True,1.44705,1.44705,True,1.44705,1.44705,True,1.154671,-0.435594
2,2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,0.0,...,1.44980,True,1.44980,1.44980,True,1.44980,1.44980,True,1.008009,-0.579950
3,3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,0.0,...,1.45310,True,1.45310,1.45310,True,1.45310,1.45310,True,0.494573,-1.085314
4,4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,0.0,...,1.45570,True,1.45570,1.45570,True,1.47590,1.43550,True,0.418697,-1.159997


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR2_SMALL.csv")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,supertrend_lower_14_2_10,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,0.0,0.0,...,1.44510,True,1.44510,1.44510,True,1.44510,1.44510,True,1.203652,-0.387382
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,0.0,0.0,...,1.44705,True,1.44705,1.44705,True,1.44705,1.44705,True,1.154671,-0.435594
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,0.0,0.0,...,1.44980,True,1.44980,1.44980,True,1.44980,1.44980,True,1.008009,-0.579950
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,0.0,0.0,...,1.45310,True,1.45310,1.45310,True,1.45310,1.45310,True,0.494573,-1.085314
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,0.0,0.0,...,1.45570,True,1.45570,1.45570,True,1.47590,1.43550,True,0.418697,-1.159997


In [ ]:
#2nd 계정 로드
data = pd.read_csv("/content/drive/MyDrive/SOL60_INDICATOR2_SMALL")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,supertrend_lower_14_2_10,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,0.0,0.0,...,1.44510,True,1.44510,1.44510,True,1.44510,1.44510,True,1.203652,-0.387382
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,0.0,0.0,...,1.44705,True,1.44705,1.44705,True,1.44705,1.44705,True,1.154671,-0.435594
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,0.0,0.0,...,1.44980,True,1.44980,1.44980,True,1.44980,1.44980,True,1.008009,-0.579950
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,0.0,0.0,...,1.45310,True,1.45310,1.45310,True,1.45310,1.45310,True,0.494573,-1.085314
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,0.0,0.0,...,1.45570,True,1.45570,1.45570,True,1.47590,1.43550,True,0.418697,-1.159997


######################

1/3 SMALL 저장

######################

In [ ]:
# 데이터의 첫 번째 1/3만 사용
n_samples = len(data)
data = data.iloc[:n_samples // 3]

In [ ]:
# Load the dataset from Kaggle
path2 = "/content/drive/MyDrive/Data/SOL60_INDICATOR3_SMALL"
data.to_csv(path2, index=False);

######################

지표 목록 파악

######################



In [ ]:
data.columns

Index(['Unnamed: 0', 'open_time', 'open', 'high', 'low', 'close', 'volume',
       'time', 'atr_5', 'atr_10', 'atr_14', 'atr_20', 'atr_50', 'vwap',
       'stoch_%k_14_3', 'stoch_%d_14_3', 'stoch_%k_21_5', 'stoch_%d_21_5',
       'stoch_%k_9_3', 'stoch_%d_9_3', 'stoch_%k_5_2', 'stoch_%d_5_2',
       'stoch_%k_20_7', 'stoch_%d_20_7', 'obv', 'bollinger_hband_10',
       'bollinger_lband_10', 'bollinger_hband_20', 'bollinger_lband_20',
       'bollinger_hband_50', 'bollinger_lband_50', 'bollinger_hband_100',
       'bollinger_lband_100', 'bollinger_hband_200', 'bollinger_lband_200',
       'ichimoku_base_9', 'ichimoku_conversion_9', 'ichimoku_base_26',
       'ichimoku_conversion_26', 'ichimoku_base_52', 'ichimoku_conversion_52',
       'ichimoku_base_100', 'ichimoku_conversion_100', 'ichimoku_base_200',
       'ichimoku_conversion_200', 'atr', 'upperband', 'lowerband',
       'in_uptrend', 'supertrend_upper_7_3_14', 'supertrend_lower_7_3_14',
       'supertrend_in_uptrend_7_3_14', 'super

######################

**불필요 지표 삭제**
######################

In [ ]:
data = data.drop(columns=['ichimoku_base_9',
       'ichimoku_conversion_9', 'ichimoku_base_26', 'ichimoku_conversion_26',
       'ichimoku_base_52', 'ichimoku_conversion_52', 'ichimoku_base_100',
       'ichimoku_conversion_100', 'ichimoku_base_200',
       'ichimoku_conversion_200','bollinger_hband_10', 'bollinger_lband_10',
       'bollinger_hband_20', 'bollinger_lband_20', 'bollinger_hband_50',
       'bollinger_lband_50', 'bollinger_hband_100', 'bollinger_lband_100',
       'bollinger_hband_200', 'bollinger_lband_200', 'obv'])

In [ ]:
data.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'time', 'atr_5',
       'atr_10', 'atr_14', 'atr_20', 'atr_50', 'vwap', 'stoch_%k_14_3',
       'stoch_%d_14_3', 'stoch_%k_21_5', 'stoch_%d_21_5', 'stoch_%k_9_3',
       'stoch_%d_9_3', 'stoch_%k_5_2', 'stoch_%d_5_2', 'stoch_%k_20_7',
       'stoch_%d_20_7', 'atr', 'upperband', 'lowerband', 'in_uptrend',
       'supertrend_upper_7_3_14', 'supertrend_lower_7_3_14',
       'supertrend_in_uptrend_7_3_14', 'supertrend_upper_10_3_20',
       'supertrend_lower_10_3_20', 'supertrend_in_uptrend_10_3_20',
       'supertrend_upper_14_2_10', 'supertrend_lower_14_2_10',
       'supertrend_in_uptrend_14_2_10', 'supertrend_upper_20_4_50',
       'supertrend_lower_20_4_50', 'supertrend_in_uptrend_20_4_50',
       'supertrend_upper_50_5_5', 'supertrend_lower_50_5_5',
       'supertrend_in_uptrend_50_5_5', 'max_return_60min', 'min_return_60min'],
      dtype='object')

######################

**추가 지표 처리**
######################

In [ ]:
# 지표 계산 함수 정의

def calculate_indicators(df):
    # Parabolic SAR
    df['Parabolic_SAR_0.02'] = pata.psar(df['high'], df['low'], df['close'], af=0.02, max_af=0.2)['PSARl_0.02_0.2']
    df['Parabolic_SAR_0.04'] = pata.psar(df['high'], df['low'], df['close'], af=0.04, max_af=0.2)['PSARl_0.04_0.2']
    df['Parabolic_SAR_0.06'] = pata.psar(df['high'], df['low'], df['close'], af=0.06, max_af=0.2)['PSARl_0.06_0.2']
    df['Parabolic_SAR_0.08'] = pata.psar(df['high'], df['low'], df['close'], af=0.08, max_af=0.2)['PSARl_0.08_0.2']
    df['Parabolic_SAR_0.1'] = pata.psar(df['high'], df['low'], df['close'], af=0.1, max_af=0.2)['PSARl_0.1_0.2']

    # Williams %R
    df['Williams_%R_10'] = pata.willr(df['high'], df['low'], df['close'], length=10)
    df['Williams_%R_20'] = pata.willr(df['high'], df['low'], df['close'], length=20)
    df['Williams_%R_30'] = pata.willr(df['high'], df['low'], df['close'], length=30)
    df['Williams_%R_40'] = pata.willr(df['high'], df['low'], df['close'], length=40)
    df['Williams_%R_50'] = pata.willr(df['high'], df['low'], df['close'], length=50)

    # Momentum
    df['Momentum_10'] = pata.mom(df['close'], length=10)
    df['Momentum_20'] = pata.mom(df['close'], length=20)
    df['Momentum_30'] = pata.mom(df['close'], length=30)
    df['Momentum_40'] = pata.mom(df['close'], length=40)
    df['Momentum_50'] = pata.mom(df['close'], length=50)

    # Rate of Change (ROC)
    df['ROC_10'] = pata.roc(df['close'], length=10)
    df['ROC_20'] = pata.roc(df['close'], length=20)
    df['ROC_30'] = pata.roc(df['close'], length=30)
    df['ROC_40'] = pata.roc(df['close'], length=40)
    df['ROC_50'] = pata.roc(df['close'], length=50)

    # Chande Momentum Oscillator (CMO)
    df['CMO_10'] = pata.cmo(df['close'], length=10)
    df['CMO_20'] = pata.cmo(df['close'], length=20)
    df['CMO_30'] = pata.cmo(df['close'], length=30)
    df['CMO_40'] = pata.cmo(df['close'], length=40)
    df['CMO_50'] = pata.cmo(df['close'], length=50)

    # Money Flow Index (MFI)
    df['MFI_10'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=10)
    df['MFI_20'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=20)
    df['MFI_30'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=30)
    df['MFI_40'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=40)
    df['MFI_50'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=50)

    # Relative Strength Index (RSI)
    df['RSI_10'] = pata.rsi(df['close'], length=10)
    df['RSI_20'] = pata.rsi(df['close'], length=20)
    df['RSI_30'] = pata.rsi(df['close'], length=30)
    df['RSI_40'] = pata.rsi(df['close'], length=40)
    df['RSI_50'] = pata.rsi(df['close'], length=50)

    # Accumulation/Distribution Line (A/D Line)
    df['Accumulation_Distribution_Line'] = pata.ad(df['high'], df['low'], df['close'], df['volume'])

    # Elder's Force Index (EFI)
    df['Elder_Force_Index_2'] = pata.efi(df['close'], df['volume'], length=2)
    df['Elder_Force_Index_13'] = pata.efi(df['close'], df['volume'], length=13)
    df['Elder_Force_Index_5'] = pata.efi(df['close'], df['volume'], length=5)
    df['Elder_Force_Index_10'] = pata.efi(df['close'], df['volume'], length=10)
    df['Elder_Force_Index_25'] = pata.efi(df['close'], df['volume'], length=25)

    # Relative Vigor Index (RVI)
    df['Relative_Vigor_Index_10'] = pata.rsi(df['close'], length=10)  # pandas-ta는 RVI가 없으므로 RSI로 대체
    df['Relative_Vigor_Index_20'] = pata.rsi(df['close'], length=20)
    df['Relative_Vigor_Index_30'] = pata.rsi(df['close'], length=30)
    df['Relative_Vigor_Index_40'] = pata.rsi(df['close'], length=40)
    df['Relative_Vigor_Index_50'] = pata.rsi(df['close'], length=50)

    # Volume Ratio (VR)
    def volume_ratio(close, volume, period):
        vr = []
        for i in range(len(close)):
            if i < period:
                vr.append(None)
            else:
                vol_up = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] > close[j - 1])
                vol_down = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] < close[j - 1])
                vol_same = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] == close[j - 1])
                vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
                vr.append(vr_value)
        return vr

    df['VR_10'] = volume_ratio(df['close'], df['volume'], period=10)
    df['VR_20'] = volume_ratio(df['close'], df['volume'], period=20)
    df['VR_30'] = volume_ratio(df['close'], df['volume'], period=30)
    df['VR_40'] = volume_ratio(df['close'], df['volume'], period=40)
    df['VR_50'] = volume_ratio(df['close'], df['volume'], period=50)

    # Commodity Channel Index (CCI)
    df['CCI_10'] = pata.cci(df['high'], df['low'], df['close'], length=10)
    df['CCI_20'] = pata.cci(df['high'], df['low'], df['close'], length=20)
    df['CCI_30'] = pata.cci(df['high'], df['low'], df['close'], length=30)
    df['CCI_40'] = pata.cci(df['high'], df['low'], df['close'], length=40)
    df['CCI_50'] = pata.cci(df['high'], df['low'], df['close'], length=50)

    return df

# 지표 계산
data = calculate_indicators(data)

<ipython-input-10-3a7eadde72c6>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-10-3a7eadde72c6>:80: RuntimeWarning: invalid value encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-10-3a7eadde72c6>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-10-3a7eadde72c6>:80: RuntimeWarning: invalid value encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-10-3a7eadde72c6>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-10-3a7eadde72c6>:80: RuntimeWarning: invalid value encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<

In [ ]:
# 이격도(Price Disparity Index) 계산 함수
def calculate_disparity_index(df, period):
    df[f'disparity_index_{period}'] = (df['close'] / df['close'].rolling(window=period).mean()) * 100
    return df

# 이평선(이동평균선) 계산 함수
def calculate_moving_averages(df, periods):
    for period in periods:
        df[f'price_ma_{period}'] = df['close'].rolling(window=period).mean()
        df[f'volume_ma_{period}'] = df['volume'].rolling(window=period).mean()
    return df

# 이격도와 이동평균선 계산
disparity_periods = [5, 10, 20, 50, 100, 200]
moving_average_periods = [5, 10, 20, 50, 100, 200]

# 이격도 계산
for period in disparity_periods:
    data = calculate_disparity_index(data, period)

# 이동평균선 계산
data = calculate_moving_averages(data, moving_average_periods)

In [ ]:
data.head()

,Unnamed: 0.1,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,...,price_ma_10,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200
0,0,0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'open_time', 'open', 'high', 'low',
       'close', 'volume', 'time', 'atr_5',
       ...
       'price_ma_10', 'volume_ma_10', 'price_ma_20', 'volume_ma_20',
       'price_ma_50', 'volume_ma_50', 'price_ma_100', 'volume_ma_100',
       'price_ma_200', 'volume_ma_200'],
      dtype='object', length=141)

######################

**지표 저장**
######################

In [ ]:
path_edit = "/content/drive/MyDrive/Data/SOL60_INDICATOR3";
data.to_csv(path_edit, index=True);

######################

**지표 관계 파악(간단)**
######################

In [ ]:
# max_return_60min이 1.1 이상인 데이터 필터링
filtered_df_relative = data[data['max_return_60min'] >= 1.1]

# 'time' 컬럼을 분 단위로 변환
filtered_df_relative['time'] = filtered_df_relative['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

# 상관계수 계산 (open_time을 제외한 나머지 항목들에 대해서만 계산)
correlation_matrix = filtered_df_relative.drop(columns=['open_time']).corr()

# max_return_60min과 나머지 항목들 간의 상관계수 출력
max_return_correlations = correlation_matrix['max_return_60min'].drop('max_return_60min').sort_values(ascending=False)
max_return_correlations

<ipython-input-18-97ae0006df35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_relative['time'] = filtered_df_relative['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))


volume_ma_100                     0.337089
volume_ma_200                     0.334000
volume_ma_50                      0.320732
volume_ma_20                      0.305341
volume_ma_10                      0.291334
                                    ...   
lowerband                        -0.110123
supertrend_lower_50_5_5          -0.110123
Accumulation_Distribution_Line   -0.110828
obv                              -0.113027
Unnamed: 0                       -0.141447
Name: max_return_60min, Length: 138, dtype: float64

######################

**지표 관계 파악(복잡)**
######################

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# 목표 변수 정의
data['target'] = (data['max_return_60min'] >= 1.2).astype(int)

# target 컬럼과 max_return_60min 컬럼 제거 (목표 변수를 제외하고 피처 선택을 위해)
data_features = data.drop(columns=['max_return_60min', 'target', 'Unnamed: 0.1', 'min_return_60min' ])

# 무한대 값을 NaN으로 변환
data_features.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값 처리
imputer = SimpleImputer(strategy='mean')
data_features_imputed = imputer.fit_transform(data_features)

# 다시 데이터프레임으로 변환
data_features_imputed = pd.DataFrame(data_features_imputed, columns=data_features.columns)

# 목표 변수 추가
data_features_imputed['target'] = data['target']

# 데이터 분할
X = data_features_imputed.drop('target', axis=1)
y = data_features_imputed['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 변수 중요도 추출
feature_importances = model.feature_importances_
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importances
})

# 중요도 순으로 정렬
importance_df = importance_df.sort_values(by='importance', ascending=False)

# 변수 중요도 출력
print("\n랜덤 포레스트 변수 중요도:\n", importance_df)

# 중요도가 낮은 변수 제거
importance_threshold = 0.007   # 중요도 임계값 설정
relevant_features_importance = importance_df[importance_df['importance'] > importance_threshold]['feature']

# 관계성이 적은 변수 제거
final_filtered_data = data_features_imputed[relevant_features_importance]

# 최종 필터링 결과 출력
print("\n랜덤 포레스트 변수 중요도에 의한 유의미한 피처들:\n", relevant_features_importance)


랜덤 포레스트 변수 중요도:
                             feature  importance
22                              obv    0.038461
135                   volume_ma_200    0.031499
97   Accumulation_Distribution_Line    0.026909
133                   volume_ma_100    0.026399
131                    volume_ma_50    0.025150
..                              ...         ...
46                       in_uptrend    0.000997
61     supertrend_in_uptrend_50_5_5    0.000971
52    supertrend_in_uptrend_10_3_20    0.000729
49     supertrend_in_uptrend_7_3_14    0.000704
55    supertrend_in_uptrend_14_2_10    0.000478

[136 rows x 2 columns]

랜덤 포레스트 변수 중요도에 의한 유의미한 피처들:
 22                                obv
135                     volume_ma_200
97     Accumulation_Distribution_Line
133                     volume_ma_100
131                      volume_ma_50
5                                time
10                             atr_50
9                              atr_20
129                      volume_ma_20
123      

In [ ]:
path_edit = "/content/drive/MyDrive/Data/SOL60_INDICATOR3_importance";
final_filtered_data.to_csv(path_edit, index=True);

NameError: name 'final_filtered_data' is not defined

In [ ]:
final_filtered_data

,obv,volume_ma_200,Accumulation_Distribution_Line,volume_ma_100,volume_ma_50,time,atr_50,atr_20,volume_ma_20,disparity_index_200,...,atr_14,price_ma_200,ichimoku_conversion_200,bollinger_lband_100,atr_10,bollinger_hband_100,disparity_index_100,price_ma_100,ichimoku_conversion_100,volume_ma_5
0,1.613400e+02,3370.005505,1.152429e+02,3369.908842,3369.844991,900.0,0.000000,0.000000,3369.803441,100.029344,...,0.000000,68.32352,68.29462,67.597513,0.000000,69.049791,100.014685,68.323652,68.305657,3369.784317
1,3.102000e+02,3370.005505,-3.361714e+01,3369.908842,3369.844991,901.0,0.000000,0.000000,3369.803441,100.029344,...,0.000000,68.32352,68.29462,67.597513,0.000000,69.049791,100.014685,68.323652,68.305657,3369.784317
2,3.518330e+03,3370.005505,-1.437174e+03,3369.908842,3369.844991,902.0,0.000000,0.000000,3369.803441,100.029344,...,0.000000,68.32352,68.29462,67.597513,0.000000,69.049791,100.014685,68.323652,68.305657,3369.784317
3,5.158090e+03,3370.005505,1.440231e+02,3369.908842,3369.844991,903.0,0.000000,0.000000,3369.803441,100.029344,...,0.000000,68.32352,68.29462,67.597513,0.000000,69.049791,100.014685,68.323652,68.305657,3369.784317
4,6.059000e+03,3370.005505,1.044933e+03,3369.908842,3369.844991,904.0,0.000000,0.000000,3369.803441,100.029344,...,0.000000,68.32352,68.29462,67.597513,0.000000,69.049791,100.014685,68.323652,68.305657,1211.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856524,2.658873e+08,757.289600,3.849234e+07,669.864430,757.371700,896.0,0.108608,0.103686,722.919150,99.980145,...,0.101958,140.01780,139.93500,139.407271,0.101579,140.365529,100.074060,139.886400,139.870000,843.728400
1856525,2.658873e+08,756.820090,3.849227e+07,653.455910,746.778960,897.0,0.108236,0.103002,635.712150,99.916367,...,0.101104,140.01710,139.93500,139.406985,0.100421,140.362415,100.010938,139.884700,139.870000,622.814600
1856526,2.658874e+08,755.148725,3.849224e+07,652.140930,741.989660,898.0,0.107071,0.100352,589.633950,99.916974,...,0.097454,140.01625,139.93500,139.406858,0.095379,140.358742,100.012296,139.882800,139.870000,506.389600
1856527,2.658879e+08,756.760605,3.849210e+07,653.667760,740.509560,899.0,0.106129,0.098334,605.142000,99.924794,...,0.094778,140.01530,139.93500,139.406524,0.091841,140.356876,100.020231,139.881700,139.870000,465.279800


In [ ]:
# 중요도가 낮은 변수 제거
importance_threshold = 0.007  # 중요도 임계값 설정
relevant_features_importance = importance_df[importance_df['importance'] > importance_threshold]['feature']

NameError: name 'importance_df' is not defined

In [ ]:
relevant_features_importance.head(60)

22                                obv
135                     volume_ma_200
97     Accumulation_Distribution_Line
133                     volume_ma_100
131                      volume_ma_50
5                                time
10                             atr_50
9                              atr_20
129                      volume_ma_20
123               disparity_index_200
32                bollinger_lband_200
127                      volume_ma_10
31                bollinger_hband_200
8                              atr_14
134                      price_ma_200
42            ichimoku_conversion_200
30                bollinger_lband_100
7                              atr_10
29                bollinger_hband_100
122               disparity_index_100
132                      price_ma_100
40            ichimoku_conversion_100
125                       volume_ma_5
28                 bollinger_lband_50
27                 bollinger_hband_50
112                             VR_50
130         

In [ ]:
final_filtered_data = data_features_imputed[relevant_features_importance]

######################

**학습 데이터 처리(공통)**
######################

In [23]:
# open_time 열이 datetime 형식이 아닌 경우 변환
if not np.issubdtype(data['open_time'].dtype, np.datetime64):
    data['open_time'] = pd.to_datetime(data['open_time'])

# time 열을 분 단위로 변환
data['time'] = data['open_time'].dt.hour * 60 + data['open_time'].dt.minute

# 사용하지 않을 열 제외
data = data.drop(columns=['open_time', 'Unnamed: 0'])

In [24]:
data.columns

Index(['Unnamed: 0.1', 'open', 'high', 'low', 'close', 'volume', 'time',
       'atr_5', 'atr_10', 'atr_14',
       ...
       'price_ma_10', 'volume_ma_10', 'price_ma_20', 'volume_ma_20',
       'price_ma_50', 'volume_ma_50', 'price_ma_100', 'volume_ma_100',
       'price_ma_200', 'volume_ma_200'],
      dtype='object', length=139)

In [25]:
# 사용하지 않을 열 제외
data = data.drop(columns=['Unnamed: 0.1'])

In [ ]:
data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'time', 'atr_5', 'atr_10',
       'atr_14', 'atr_20',
       ...
       'price_ma_10', 'volume_ma_10', 'price_ma_20', 'volume_ma_20',
       'price_ma_50', 'volume_ma_50', 'price_ma_100', 'volume_ma_100',
       'price_ma_200', 'volume_ma_200'],
      dtype='object', length=138)

######################

**학습 데이터 처리(시계열)**
######################

In [ ]:
# 목표 변수 생성 (max_return_60min이 1.1% 이상인 경우 1, 그렇지 않으면 0)
data['target'] = (data['max_return_60min'] >= 1.1).astype(int)

# 특성과 목표 변수 분리
X = data.drop(columns=['max_return_60min', 'min_return_60min', 'target'])
y = data['target']

# 무한대 값을 NaN으로 대체
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균으로 대체
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_imputed)

# 시계열 데이터 형태로 변환 (LSTM용)
def create_sequences(data, target, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        label = target[i + sequence_length]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

# 시퀀스 길이 설정
sequence_length = 60

# LSTM용 데이터 생성
X_seq, y_seq = create_sequences(X_scaled, y, sequence_length)

######################

**학습 데이터 처리(비시계열)**
######################

In [ ]:
# 남길 컬럼 리스트
selected_columns = [
    "obv", "volume_ma_200", "Accumulation_Distribution_Line", "volume_ma_100", "volume_ma_50", "time",
    "atr_50", "atr_20", "volume_ma_20", "disparity_index_200", "bollinger_lband_200", "volume_ma_10",
    "bollinger_hband_200", "atr_14", "price_ma_200", "ichimoku_conversion_200", "bollinger_lband_100",
    "atr_10", "bollinger_hband_100", "disparity_index_100", "price_ma_100", "ichimoku_conversion_100",
    "volume_ma_5", "bollinger_lband_50", "bollinger_hband_50", "VR_50", "price_ma_50", "ichimoku_conversion_52",
    "atr_5", "MFI_50", "ROC_50", "supertrend_lower_20_4_50", "atr", "VR_40", "supertrend_lower_50_5_5", "MFI_40",
    "Elder_Force_Index_25", "lowerband", "ROC_40", "bollinger_lband_20", "supertrend_upper_20_4_50", "VR_30",
    "supertrend_lower_7_3_14"
]

# 선택된 컬럼만으로 데이터프레임 생성
filtered_data = data[selected_columns]

# 목표 변수 생성 (max_return_60min이 1.1% 이상인 경우 1, 그렇지 않으면 0)
data['target'] = (data['max_return_60min'] >= 1.2).astype(int)

# 목표 변수 추가
filtered_data['target'] = data['target']

# 데이터 분할
X = filtered_data.drop('target', axis=1)
y = filtered_data['target']

<ipython-input-9-dea2a10ee53a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['target'] = data['target']


In [26]:
# 목표 변수 생성 (max_return_60min이 1.1% 이상인 경우 1, 그렇지 않으면 0)
data['target'] = (data['max_return_60min'] >= 1.1).astype(int)

# 특성과 목표 변수 분리
X = data.drop(columns=['max_return_60min', 'min_return_60min', 'target'])
y = data['target']

In [27]:
# 무한대 값을 NaN으로 대체
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균으로 대체
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_imputed)

######################

**xgboost**
######################

In [ ]:
pip install xgboost

In [ ]:
import xgboost as xgb

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# XGBoost 모델 정의
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42, use_label_encoder=False, eval_metric='logloss')

# 모델 학습
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Accuracy: 0.697
Precision: 0.670
Recall: 0.420
F1 Score: 0.516


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_xgb_model.joblib'
joblib.dump(xgb_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_xgb_model.joblib


In [ ]:
# 모델 로딩
model_path = '/content/drive/MyDrive/SO60L_SMALL_INDICATOR3_xgb_model.joblib'

xgb_model = joblib.load(model_path)

print("Model loaded successfully")

In [ ]:
# 예측 수행
predictions = xgb_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_xgb'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_xgb'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_xgb
7109,7109,2024-07-19 13:29:00,162.75,162.80,162.59,162.63,1289.717,809,0.214352,0.235240,...,False,163.590931,161.769140,False,163.766759,161.623241,False,3.818484,-0.215212,0
2880,2880,2024-07-16 15:00:00,155.91,155.93,155.69,155.69,1983.029,900,0.219039,0.222679,...,True,156.862755,154.977555,True,156.426560,154.714806,False,3.796005,-0.057807,1
2881,2881,2024-07-16 15:01:00,155.70,155.78,155.60,155.71,1419.077,901,0.211231,0.218411,...,True,156.736100,154.977555,True,156.426560,154.633844,False,3.782673,-0.070644,1
7106,7106,2024-07-19 13:26:00,162.82,162.85,162.69,162.70,1820.981,806,0.233500,0.250055,...,False,163.704624,161.835376,False,163.937498,161.602502,False,3.773817,-0.258144,0
7107,7107,2024-07-19 13:27:00,162.69,162.78,162.53,162.72,2155.233,807,0.236800,0.250049,...,False,163.590931,161.719069,False,163.838998,161.471002,False,3.761062,-0.270403,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,8522,2024-07-20 13:02:00,168.56,168.73,168.56,168.73,846.118,782,0.132566,0.128740,...,True,168.816198,168.145976,False,169.307830,168.249485,True,0.000000,-0.865288,0
3519,3519,2024-07-17 01:39:00,163.03,163.15,163.03,163.15,619.675,99,0.219354,0.251539,...,True,164.098585,162.673690,True,164.186768,162.223585,True,0.000000,-1.121667,0
1075,1075,2024-07-15 08:55:00,152.72,153.06,152.72,153.06,1988.077,535,0.266985,0.238983,...,False,153.399907,152.140127,False,153.993659,151.555073,False,0.000000,-0.672939,1
7594,7594,2024-07-19 21:34:00,169.14,169.29,169.14,169.29,1151.051,1294,0.174188,0.174318,...,False,169.778412,168.489909,False,170.016173,168.344062,False,0.000000,-0.886054,0


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1) & (data_test['prediction_xgb'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1) & (data_test['prediction_xgb'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 16.734174285383634
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 25.911352852679087


######################

**catboost**
######################

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# CatBoost 모델 정의
cat_model = CatBoostClassifier(iterations=100, learning_rate=0.05, depth=6, random_seed=42, verbose=0)

# 모델 학습
cat_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred = cat_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Accuracy: 0.680
Precision: 0.641
Recall: 0.384
F1 Score: 0.480


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(cat_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_cat_model.joblib'
joblib.dump(cat_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_cat_model.joblib


In [ ]:
# ExtraTreesClassifier 모델 로딩
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_cat_model.joblib'

cat_model = joblib.load(model_path_load)

print("Model loaded successfully")

In [ ]:
# 예측 수행
predictions = cat_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_cat'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_cat'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_xgb,prediction_cat
7109,7109,2024-07-19 13:29:00,162.75,162.80,162.59,162.63,1289.717,809,0.214352,0.235240,...,163.590931,161.769140,False,163.766759,161.623241,False,3.818484,-0.215212,0,1
2880,2880,2024-07-16 15:00:00,155.91,155.93,155.69,155.69,1983.029,900,0.219039,0.222679,...,156.862755,154.977555,True,156.426560,154.714806,False,3.796005,-0.057807,1,1
2881,2881,2024-07-16 15:01:00,155.70,155.78,155.60,155.71,1419.077,901,0.211231,0.218411,...,156.736100,154.977555,True,156.426560,154.633844,False,3.782673,-0.070644,1,1
7106,7106,2024-07-19 13:26:00,162.82,162.85,162.69,162.70,1820.981,806,0.233500,0.250055,...,163.704624,161.835376,False,163.937498,161.602502,False,3.773817,-0.258144,0,1
7107,7107,2024-07-19 13:27:00,162.69,162.78,162.53,162.72,2155.233,807,0.236800,0.250049,...,163.590931,161.719069,False,163.838998,161.471002,False,3.761062,-0.270403,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,8522,2024-07-20 13:02:00,168.56,168.73,168.56,168.73,846.118,782,0.132566,0.128740,...,168.816198,168.145976,False,169.307830,168.249485,True,0.000000,-0.865288,0,0
3519,3519,2024-07-17 01:39:00,163.03,163.15,163.03,163.15,619.675,99,0.219354,0.251539,...,164.098585,162.673690,True,164.186768,162.223585,True,0.000000,-1.121667,0,1
1075,1075,2024-07-15 08:55:00,152.72,153.06,152.72,153.06,1988.077,535,0.266985,0.238983,...,153.399907,152.140127,False,153.993659,151.555073,False,0.000000,-0.672939,1,1
7594,7594,2024-07-19 21:34:00,169.14,169.29,169.14,169.29,1151.051,1294,0.174188,0.174318,...,169.778412,168.489909,False,170.016173,168.344062,False,0.000000,-0.886054,0,0


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1) & (data_test['prediction_cat'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1) & (data_test['prediction_cat'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 15.542182617752577
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 28.503645411410716


######################

**lightgbm**
######################

In [ ]:
pip install lightgbm

In [ ]:
import lightgbm as lgb

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# LightGBM 데이터셋 생성
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train, params={'max_bin': 255})
lgb_eval = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test, reference=lgb_train, params={'max_bin': 255})

# LightGBM 하이퍼파라미터 설정
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

# 모델 학습
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=100,
    valid_sets=lgb_eval,
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# 예측
y_pred_prob = gbm.predict(X_test.reshape(X_test.shape[0], -1), num_iteration=gbm.best_iteration)
y_pred = (y_pred_prob > 0.5).astype(int)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

X_train shape: (495074, 136)
X_test shape: (123769, 136)
y_train shape: (495074,)
y_test shape: (123769,)
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.571114	valid_0's auc: 0.753208
Accuracy: 0.6998925417511654
Precision: 0.6690122906450997
Recall: 0.439928810720268
F1 Score: 0.5308086804941515


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(gbm)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_INDICATOR3_LightGBM_model.joblib'
joblib.dump(gbm, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_INDICATOR3_LightGBM_model.joblib


In [ ]:
#모델 로딩
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_INDICATOR3_LightGBM_model.joblib'

gbm = joblib.load(model_path)

print("Model loaded successfully")

Model loaded successfully


In [ ]:
# 예측 수행
predictions_prob = gbm.predict(data_test_predict_scaled)

# 확률을 0 또는 1로 변환 (기본 임계값 0.5)
predictions = (predictions_prob > 0.5).astype(int)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_LightGBM'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_LightGBM'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200,prediction_LightGBM
1330056,2023-07-14 00:28:00,25.71,25.73,25.69,25.70,9232.62,28,0.093703,0.091990,0.089117,...,9979.665,25.8620,12290.3035,25.9018,9497.1206,25.7946,8068.2889,25.72095,6753.54055,1
1330054,2023-07-14 00:26:00,25.79,25.79,25.64,25.70,13705.60,26,0.098910,0.093075,0.089272,...,9258.745,25.8750,11943.9125,25.9010,9109.5412,25.7946,7875.7339,25.71915,6900.80405,1
1330055,2023-07-14 00:27:00,25.70,25.74,25.60,25.72,17370.59,27,0.107128,0.097767,0.092896,...,10001.130,25.8690,12206.5100,25.9018,9382.8334,25.7947,8022.9095,25.72010,6834.58885,1
1330053,2023-07-14 00:25:00,25.90,25.91,25.78,25.78,5433.67,25,0.086138,0.086750,0.084601,...,9272.164,25.8815,11642.2380,25.9010,8971.1030,25.7941,7759.9606,25.71865,6864.04160,1
1330040,2023-07-14 00:12:00,25.78,25.82,25.77,25.78,4471.04,12,0.070943,0.070237,0.068448,...,9656.427,25.9350,9380.1370,25.8228,7012.1630,25.7653,6697.6975,25.69465,6733.50400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335376,2023-07-17 17:08:00,26.63,26.69,26.62,26.66,14424.84,1028,0.074813,0.070265,0.069679,...,6736.659,26.7530,6034.5025,26.8404,7075.9258,26.9210,5827.6165,27.04440,5010.64660,1
1337589,2023-07-19 06:01:00,26.51,26.51,26.48,26.50,1205.26,361,0.035754,0.033494,0.032721,...,4165.484,26.4585,2527.8670,26.4172,2397.2814,26.3856,2333.7355,26.36760,2942.69400,1
1336954,2023-07-18 19:26:00,25.51,25.52,25.47,25.51,2907.05,1166,0.042682,0.045614,0.045902,...,3198.212,25.5880,3939.9740,25.6068,3284.9080,25.5471,2888.1443,25.44505,3840.66330,1
1346065,2023-07-25 03:17:00,23.46,23.47,23.46,23.46,281.27,197,0.017135,0.019364,0.020167,...,682.724,23.4620,699.2765,23.4606,1358.1764,23.3837,1089.6020,23.34215,1328.49020,1


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.2) & (data_test['prediction_LightGBM'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.2) & (data_test['prediction_LightGBM'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 13.831779627466096
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 25.686482678159077


######################

**RandomForestClassifier**
######################

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# # 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_rf = rf_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, zero_division=1)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest - Accuracy: {accuracy_rf:.3f}")
print(f"Random Forest - Precision: {precision_rf:.3f}")
print(f"Random Forest - Recall: {recall_rf:.3f}")
print(f"Random Forest - F1 Score: {f1_rf:.3f}")

# 교차 검증 평가
accuracy_cv = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Random Forest - Accuracy: 0.959
Random Forest - Precision: 0.956
Random Forest - Recall: 0.936
Random Forest - F1 Score: 0.946


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_RandomForestClassifier_model.joblib'
joblib.dump(rf_model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
#2번째 계정 전용
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/SO60_SMALL_RandomForestClassifierr_model2.joblib'
joblib.dump(rf_model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(rf_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# 모델 로딩
model_path_load = '/content/drive/MyDrive/Data/SO60_SMALL_RandomForestClassifierr_model2.joblib'

rf_model = joblib.load(model_path_load)

print("Model loaded successfully")

Model loaded successfully


In [ ]:
# 예측 수행
predictions = rf_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_rf'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_rf'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_xgb,prediction_cat,prediction_LightGBM,prediction_rf
7109,7109,2024-07-19 13:29:00,162.75,162.80,162.59,162.63,1289.717,809,0.214352,0.235240,...,False,163.766759,161.623241,False,3.818484,-0.215212,0,1,1,1
2880,2880,2024-07-16 15:00:00,155.91,155.93,155.69,155.69,1983.029,900,0.219039,0.222679,...,True,156.426560,154.714806,False,3.796005,-0.057807,1,1,1,1
2881,2881,2024-07-16 15:01:00,155.70,155.78,155.60,155.71,1419.077,901,0.211231,0.218411,...,True,156.426560,154.633844,False,3.782673,-0.070644,1,1,1,1
7106,7106,2024-07-19 13:26:00,162.82,162.85,162.69,162.70,1820.981,806,0.233500,0.250055,...,False,163.937498,161.602502,False,3.773817,-0.258144,0,1,1,1
7107,7107,2024-07-19 13:27:00,162.69,162.78,162.53,162.72,2155.233,807,0.236800,0.250049,...,False,163.838998,161.471002,False,3.761062,-0.270403,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,8522,2024-07-20 13:02:00,168.56,168.73,168.56,168.73,846.118,782,0.132566,0.128740,...,False,169.307830,168.249485,True,0.000000,-0.865288,0,0,0,0
3519,3519,2024-07-17 01:39:00,163.03,163.15,163.03,163.15,619.675,99,0.219354,0.251539,...,True,164.186768,162.223585,True,0.000000,-1.121667,0,1,1,1
1075,1075,2024-07-15 08:55:00,152.72,153.06,152.72,153.06,1988.077,535,0.266985,0.238983,...,False,153.993659,151.555073,False,0.000000,-0.672939,1,1,1,1
7594,7594,2024-07-19 21:34:00,169.14,169.29,169.14,169.29,1151.051,1294,0.174188,0.174318,...,False,170.016173,168.344062,False,0.000000,-0.886054,0,0,0,0


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1) & (data_test['prediction_rf'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1) & (data_test['prediction_rf'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 16.294410369170233
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 24.661497511862052


######################

**GradientBoostingClassifier**
######################

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Gradient Boosting 모델 정의
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42)

# 모델 학습
gb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_gb = gb_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_gb = accuracy_score(y_test, y_pred_gb)
precision_gb = precision_score(y_test, y_pred_gb, zero_division=1)
recall_gb = recall_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Accuracy: {accuracy_gb:.3f}")
print(f"Gradient Boosting - Precision: {precision_gb:.3f}")
print(f"Gradient Boosting - Recall: {recall_gb:.3f}")
print(f"Gradient Boosting - F1 Score: {f1_gb:.3f}")

# 교차 검증 평가
accuracy_cv = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_GradientBoostingClassifier_model.joblib'
joblib.dump(gb_model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
#2번째 계정 전용
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/SO60_SMALL_GradientBoostingClassifier_model2.joblib'
joblib.dump(gb_model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(gb_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# ExtraTreesClassifier 모델 로딩
model_path_load = '/content/drive/MyDrive/Data/SO60_SMALL_GradientBoostingClassifier_model.joblib'

gb_model = joblib.load(model_path_load)

print("Model loaded successfully")

Model loaded successfully


In [ ]:
# 예측 수행
predictions = gb_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_gb'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_gb'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_gb
1330056,1330056,2023-07-14 00:28:00,25.71,25.73,25.69,25.70,9232.62,28,0.093703,0.091990,...,False,25.953513,25.428957,False,26.178513,25.241487,False,25.019455,0.000000,1
1330054,1330054,2023-07-14 00:26:00,25.79,25.79,25.64,25.70,13705.60,26,0.098910,0.093075,...,False,25.992870,25.437130,False,26.209552,25.220448,False,25.019455,-0.389105,1
1330055,1330055,2023-07-14 00:27:00,25.70,25.74,25.60,25.72,17370.59,27,0.107128,0.097767,...,False,25.953513,25.386487,False,26.205642,25.134358,False,24.922240,-0.116641,1
1330053,1330053,2023-07-14 00:25:00,25.90,25.91,25.78,25.78,5433.67,25,0.086138,0.086750,...,False,26.116296,25.573704,False,26.275690,25.727413,True,24.631497,-0.698216,1
1330040,1330040,2023-07-14 00:12:00,25.78,25.82,25.77,25.78,4471.04,12,0.070943,0.070237,...,False,26.009573,25.561118,False,26.149717,25.727413,True,24.631497,-0.698216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335376,1335376,2023-07-17 17:08:00,26.63,26.69,26.62,26.66,14424.84,1028,0.074813,0.070265,...,False,26.913115,26.396885,False,26.991015,26.280934,False,-0.075019,-3.038260,1
1337589,1337589,2023-07-19 06:01:00,26.51,26.51,26.48,26.50,1205.26,361,0.035754,0.033494,...,True,26.633162,26.364355,True,26.673772,26.355043,True,-0.075472,-1.320755,1
1336954,1336954,2023-07-18 19:26:00,25.51,25.52,25.47,25.51,2907.05,1166,0.042682,0.045614,...,False,25.681383,25.308387,False,25.704786,25.281590,False,-0.078401,-1.019208,1
1346065,1346065,2023-07-25 03:17:00,23.46,23.47,23.46,23.46,281.27,197,0.017135,0.019364,...,False,23.553539,23.417285,True,23.550674,23.399324,True,-0.085251,-0.596760,1


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1) & (data_test['prediction_gb'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1) & (data_test['prediction_gb'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 17.787710291624855
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 21.013892053396418


######################

**AdaBoostClassifier**
######################

In [ ]:
pip install pytorch-tabnet optuna

In [ ]:
import optuna
from sklearn.ensemble import AdaBoostClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Optuna를 사용한 하이퍼파라미터 튜닝
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0, log=True)

    ab_model = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    ab_model.fit(X_train, y_train)
    y_pred = ab_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# 최적 하이퍼파라미터 출력
best_params = study.best_trial.params
print("Best hyperparameters found:")
for key, value in best_params.items():
    print(f"{key}: {value}")

In [ ]:
# 최적 하이퍼파라미터로 모델 학습
ab_model = AdaBoostClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], random_state=42)
ab_model.fit(X_train, y_train)

# 예측
y_pred_ab = ab_model.predict(X_test)

# 평가
accuracy_ab = accuracy_score(y_test, y_pred_ab)
precision_ab = precision_score(y_test, y_pred_ab, zero_division=1)
recall_ab = recall_score(y_test, y_pred_ab)
f1_ab = f1_score(y_test, y_pred_ab)

print(f"AdaBoost - Accuracy: {accuracy_ab:.3f}")
print(f"AdaBoost - Precision: {precision_ab:.3f}")
print(f"AdaBoost - Recall: {recall_ab:.3f}")
print(f"AdaBoost - F1 Score: {f1_ab:.3f}")

In [ ]:
#임시 파라미
from sklearn.ensemble import AdaBoostClassifier

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# AdaBoost 모델 정의
ab_model = AdaBoostClassifier(n_estimators=100, learning_rate=0.05, random_state=42)

# 모델 학습
ab_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_ab = ab_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_ab = accuracy_score(y_test, y_pred_ab)
precision_ab = precision_score(y_test, y_pred_ab, zero_division=1)
recall_ab = recall_score(y_test, y_pred_ab)
f1_ab = f1_score(y_test, y_pred_ab)

print(f"AdaBoost - Accuracy: {accuracy_ab:.3f}")
print(f"AdaBoost - Precision: {precision_ab:.3f}")
print(f"AdaBoost - Recall: {recall_ab:.3f}")
print(f"AdaBoost - F1 Score: {f1_ab:.3f}")

AdaBoost - Accuracy: 0.669
AdaBoost - Precision: 0.630
AdaBoost - Recall: 0.347
AdaBoost - F1 Score: 0.447


In [ ]:
# 교차 검증 평가
accuracy_cv = cross_val_score(ab_model, X_train, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(ab_model, X_train, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(ab_model, X_train, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(ab_model, X_train, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SOL60_FULL_INDICATOR3_AdaBoostClassifier_model.joblib'
joblib.dump(ab_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SOL60_FULL_INDICATOR3_AdaBoostClassifier_model.joblib


In [ ]:
#2번째 계정 전용
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/SO60_SMALL_AdaBoostClassifier_model2.joblib'
joblib.dump(ab_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/SO60_SMALL_AdaBoostClassifier_model2.joblib


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(ab_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# 모델 로딩
model_path_load = '/content/drive/MyDrive/Data/SOL60_FULL_INDICATOR3_AdaBoostClassifier_model.joblib'

ab_model = joblib.load(model_path_load)

print("Model loaded successfully")

Model loaded successfully


In [ ]:
# 예측 수행
predictions = ab_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_ab'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_ab'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200,prediction_ab
1047753,2022-12-29 22:05:00,8.36,8.37,8.34,8.35,10561.71,1325,0.033625,0.034524,0.035585,...,8102.205,8.3880,8117.9660,8.3598,9636.0786,8.3398,21511.3793,8.61855,19199.22915,1
1047754,2022-12-29 22:06:00,8.34,8.36,8.34,8.35,1605.04,1326,0.030900,0.033071,0.034472,...,8048.333,8.3865,7848.2100,8.3602,9451.3368,8.3372,21419.9359,8.61425,19202.05880,1
1047749,2022-12-29 22:01:00,8.37,8.38,8.35,8.36,4238.04,1321,0.033849,0.034673,0.035973,...,7852.626,8.3955,8324.4860,8.3600,9897.3732,8.3482,22011.9198,8.63575,19156.98495,1
1047757,2022-12-29 22:09:00,8.37,8.38,8.36,8.36,1968.37,1329,0.025581,0.029529,0.031587,...,5951.699,8.3850,7058.8500,8.3630,9205.1186,8.3313,21116.8933,8.60175,19160.44735,1
1047735,2022-12-29 21:47:00,8.39,8.39,8.35,8.36,5033.92,1307,0.038514,0.040075,0.041471,...,8998.236,8.3555,10514.4910,8.3386,12683.3338,8.3861,23130.2088,8.69380,19024.55800,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048311,2022-12-30 07:23:00,9.26,9.27,9.23,9.25,22499.80,443,0.037005,0.034732,0.033375,...,12240.170,9.3210,8187.4865,9.3654,5662.3116,9.3803,10192.0166,9.47650,8251.54360,1
1047584,2022-12-29 19:16:00,9.09,9.09,9.04,9.07,27911.10,1156,0.034225,0.031308,0.030527,...,10691.936,9.1175,9358.6905,9.1732,8213.5380,9.2520,6533.1999,9.31975,4811.66080,1
1047633,2022-12-29 20:05:00,8.80,8.80,8.77,8.78,9039.50,1205,0.039851,0.041652,0.042575,...,8406.316,8.7850,14018.2790,8.8462,21907.8132,9.0114,14842.2311,9.19035,9394.87475,1
1047646,2022-12-29 20:18:00,8.70,8.70,8.67,8.68,7418.21,1218,0.039387,0.042609,0.043413,...,10982.462,8.7290,10722.8850,8.7854,14649.2924,8.9417,15533.1071,9.14345,9914.12480,1


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1) & (data_test['prediction_ab'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1) & (data_test['prediction_ab'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 12.297787968838623
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 57.68086574462343


######################

**ExtraTreesClassifier**
######################

In [ ]:
#신규 ExtraTreesClassifier 모델 학습 및 예측 및 결과 검증

from sklearn.ensemble import ExtraTreesClassifier

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 학습
et_model = ExtraTreesClassifier(random_state=42)
et_model.fit(X_train, y_train)

# 예측
y_pred_et = et_model.predict(X_test)

# 평가
accuracy_et = accuracy_score(y_test, y_pred_et)
precision_et = precision_score(y_test, y_pred_et, zero_division=1)
recall_et = recall_score(y_test, y_pred_et)
f1_et = f1_score(y_test, y_pred_et)

print(f"Extra Trees - Accuracy: {accuracy_et:.3f}")
print(f"Extra Trees - Precision: {precision_et:.3f}")
print(f"Extra Trees - Recall: {recall_et:.3f}")
print(f"Extra Trees - F1 Score: {f1_et:.3f}")

# 교차 검증 평가
accuracy_cv = cross_val_score(et_model, X_train, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(et_model, X_train, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(et_model, X_train, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(et_model, X_train, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

Extra Trees - Accuracy: 0.963
Extra Trees - Precision: 0.957
Extra Trees - Recall: 0.945
Extra Trees - F1 Score: 0.951
Cross-Validated Accuracy: 0.958
Cross-Validated Precision: 0.953
Cross-Validated Recall: 0.938
Cross-Validated F1 Score: 0.945


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_ExtraTreesClassifier_model.joblib'
joblib.dump(et_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_ExtraTreesClassifier_model.joblib


In [ ]:
#2번째 계정 전용
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/SO60_SMALL_ExtraTreesClassifier_model2.joblib'
joblib.dump(et_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/SO60_SMALL_ExtraTreesClassifier_model2.joblib


In [ ]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# SHAP 값 계산 및 시각화
explainer = shap.Explainer(et_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# ExtraTreesClassifier 모델 로딩
model_path_load = '/content/drive/MyDrive/Data/SO60_SMALL_ExtraTreesClassifier_model2.joblib'

et_model = joblib.load(model_path_load)

print("Model loaded successfully")


Model loaded successfully


In [ ]:
#2번째 계정 전용
# ExtraTreesClassifier 모델 로딩
model_path_load = '/content/drive/MyDrive/SO60_SMALL_ExtraTreesClassifier_model2.joblib'

et_model = joblib.load(model_path_load)

print("Model loaded successfully")

In [ ]:
# 로드된 ExtraTreesClassifier 모델 예측 및 결과 검증

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 예측
y_pred_et = et_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_et = accuracy_score(y_test, y_pred_et)
precision_et = precision_score(y_test, y_pred_et, zero_division=1)
recall_et = recall_score(y_test, y_pred_et)
f1_et = f1_score(y_test, y_pred_et)

print(f"Extra Trees - Accuracy: {accuracy_et:.3f}")
print(f"Extra Trees - Precision: {precision_et:.3f}")
print(f"Extra Trees - Recall: {recall_et:.3f}")
print(f"Extra Trees - F1 Score: {f1_et:.3f}")

# 교차 검증 평가
X_reshaped = X_train.reshape(X_train.shape[0], -1)
accuracy_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

In [ ]:
# 예측 수행
predictions = et_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_gb,prediction_ab,prediction
1330056,1330056,2023-07-14 00:28:00,25.71,25.73,25.69,25.70,9232.62,28,0.093703,0.091990,...,25.428957,False,26.178513,25.241487,False,25.019455,0.000000,1,1,1
1330054,1330054,2023-07-14 00:26:00,25.79,25.79,25.64,25.70,13705.60,26,0.098910,0.093075,...,25.437130,False,26.209552,25.220448,False,25.019455,-0.389105,1,1,1
1330055,1330055,2023-07-14 00:27:00,25.70,25.74,25.60,25.72,17370.59,27,0.107128,0.097767,...,25.386487,False,26.205642,25.134358,False,24.922240,-0.116641,1,1,1
1330053,1330053,2023-07-14 00:25:00,25.90,25.91,25.78,25.78,5433.67,25,0.086138,0.086750,...,25.573704,False,26.275690,25.727413,True,24.631497,-0.698216,1,1,1
1330040,1330040,2023-07-14 00:12:00,25.78,25.82,25.77,25.78,4471.04,12,0.070943,0.070237,...,25.561118,False,26.149717,25.727413,True,24.631497,-0.698216,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335376,1335376,2023-07-17 17:08:00,26.63,26.69,26.62,26.66,14424.84,1028,0.074813,0.070265,...,26.396885,False,26.991015,26.280934,False,-0.075019,-3.038260,1,1,1
1337589,1337589,2023-07-19 06:01:00,26.51,26.51,26.48,26.50,1205.26,361,0.035754,0.033494,...,26.364355,True,26.673772,26.355043,True,-0.075472,-1.320755,1,0,1
1336954,1336954,2023-07-18 19:26:00,25.51,25.52,25.47,25.51,2907.05,1166,0.042682,0.045614,...,25.308387,False,25.704786,25.281590,False,-0.078401,-1.019208,1,0,1
1346065,1346065,2023-07-25 03:17:00,23.46,23.47,23.46,23.46,281.27,197,0.017135,0.019364,...,23.417285,True,23.550674,23.399324,True,-0.085251,-0.596760,1,0,1


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1) & (data_test['prediction'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1) & (data_test['prediction'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 15.969963302485446
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 26.056851253064185


######################

**tensorflow**
######################

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# MLP 모델 정의
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

# 로드된 모델로 예측
y_pred_loaded = (model.predict(X_test) > 0.5).astype("int32")

# 로드된 모델 평가
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
precision_loaded = precision_score(y_test, y_pred_loaded, zero_division=1)
recall_loaded = recall_score(y_test, y_pred_loaded)
f1_loaded = f1_score(y_test, y_pred_loaded)

print(f"Loaded MLP - Accuracy: {accuracy_loaded:.3f}")
print(f"Loaded MLP - Precision: {precision_loaded:.3f}")
print(f"Loaded MLP - Recall: {recall_loaded:.3f}")
print(f"Loaded MLP - F1 Score: {f1_loaded:.3f}")

X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Epoch 1/50
15471/15471 [==============================] - 53s 3ms/step - loss: 0.6330 - accuracy: 0.6438 - val_loss: 0.6088 - val_accuracy: 0.6692
Epoch 2/50
15471/15471 [==============================] - 38s 2ms/step - loss: 0.6204 - accuracy: 0.6594 - val_loss: 0.6081 - val_accuracy: 0.6732
Epoch 3/50
15471/15471 [==============================] - 39s 3ms/step - loss: 0.6188 - accuracy: 0.6609 - val_loss: 0.6083 - val_accuracy: 0.6717
Epoch 4/50
15471/15471 [==============================] - 38s 2ms/step - loss: 0.6170 - accuracy: 0.6620 - val_loss: 0.6046 - val_accuracy: 0.6740
Epoch 5/50
15471/15471 [==============================] - 39s 2ms/step - loss: 0.6162 - accuracy: 0.6628 - val_loss: 0.6076 - val_accuracy: 0.6728
Epoch 6/50
15471/15471 [==============================] - 43s 3ms/step - loss: 0.6156 - accuracy: 0.6639 - val_loss: 0.6069 - val_accuracy: 0.6668
Epoch 7/50
154

In [ ]:
# 교차 검증 평가
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_cv = []
precision_cv = []
recall_cv = []
f1_cv = []

for train_index, test_index in kf.split(X_scaled):
    X_train_cv, X_test_cv = X_scaled[train_index], X_scaled[test_index]
    y_train_cv, y_test_cv = y[train_index], y[test_index]

    model_cv = Sequential()
    model_cv.add(Dense(64, input_dim=X_train_cv.shape[1], activation='relu'))
    model_cv.add(Dropout(0.5))
    model_cv.add(Dense(32, activation='relu'))
    model_cv.add(Dropout(0.5))
    model_cv.add(Dense(1, activation='sigmoid'))

    model_cv.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model_cv.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)

    y_pred_cv = (model_cv.predict(X_test_cv) > 0.5).astype("int32")

    accuracy_cv.append(accuracy_score(y_test_cv, y_pred_cv))
    precision_cv.append(precision_score(y_test_cv, y_pred_cv, zero_division=1))
    recall_cv.append(recall_score(y_test_cv, y_pred_cv))
    f1_cv.append(f1_score(y_test_cv, y_pred_cv))

print("MLP - 교차 검증 평가")
print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_MLP_model.joblib'
joblib.dump(model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_MLP_model.joblib


In [ ]:
#2번째 계정 전용
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/SO60_SMALL_MLP_model.joblib'
joblib.dump(model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
# 예측 수행
predictions = model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_MLP'] = predictions

19339/19339 [==============================] - 28s 1ms/step


In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_MLP'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_MLP
1330056,1330056,2023-07-14 00:28:00,25.71,25.73,25.69,25.70,9232.62,28,0.093703,0.091990,...,False,25.953513,25.428957,False,26.178513,25.241487,False,25.019455,0.000000,1.000000e+00
1330054,1330054,2023-07-14 00:26:00,25.79,25.79,25.64,25.70,13705.60,26,0.098910,0.093075,...,False,25.992870,25.437130,False,26.209552,25.220448,False,25.019455,-0.389105,1.000000e+00
1330055,1330055,2023-07-14 00:27:00,25.70,25.74,25.60,25.72,17370.59,27,0.107128,0.097767,...,False,25.953513,25.386487,False,26.205642,25.134358,False,24.922240,-0.116641,1.000000e+00
1330053,1330053,2023-07-14 00:25:00,25.90,25.91,25.78,25.78,5433.67,25,0.086138,0.086750,...,False,26.116296,25.573704,False,26.275690,25.727413,True,24.631497,-0.698216,1.000000e+00
1330040,1330040,2023-07-14 00:12:00,25.78,25.82,25.77,25.78,4471.04,12,0.070943,0.070237,...,False,26.009573,25.561118,False,26.149717,25.727413,True,24.631497,-0.698216,9.999896e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335376,1335376,2023-07-17 17:08:00,26.63,26.69,26.62,26.66,14424.84,1028,0.074813,0.070265,...,False,26.913115,26.396885,False,26.991015,26.280934,False,-0.075019,-3.038260,1.000000e+00
1337589,1337589,2023-07-19 06:01:00,26.51,26.51,26.48,26.50,1205.26,361,0.035754,0.033494,...,True,26.633162,26.364355,True,26.673772,26.355043,True,-0.075472,-1.320755,5.923811e-07
1336954,1336954,2023-07-18 19:26:00,25.51,25.52,25.47,25.51,2907.05,1166,0.042682,0.045614,...,False,25.681383,25.308387,False,25.704786,25.281590,False,-0.078401,-1.019208,4.124725e-01
1346065,1346065,2023-07-25 03:17:00,23.46,23.47,23.46,23.46,281.27,197,0.017135,0.019364,...,False,23.553539,23.417285,True,23.550674,23.399324,True,-0.085251,-0.596760,8.284887e-23


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test['prediction_MLP'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test['prediction_MLP'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 10.116620855370426
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 2.4413300303954317


######################

**Stochastic Gradient Boosting**
######################

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Stochastic Gradient Boosting 모델 생성 및 학습
sgb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,  # 각 단계에서 80%의 샘플만 사용하여 무작위성 도입
    random_state=42
)
sgb_model.fit(X_train, y_train)

# 예측 수행
y_pred_prob = sgb_model.predict_proba(X_test)[:, 1]  # 클래스 1에 대한 확률
y_pred = (y_pred_prob > 0.5).astype(int)  # 임계값 0.5를 기준으로 0 또는 1로 변환

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Accuracy: 0.684094434748885
Precision: 0.6486109658754227
Recall: 0.39071095620039475
F1 Score: 0.48766297582388785


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_sgb_model.joblib'
joblib.dump(sgb_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_sgb_model.joblib


In [ ]:
# 예측 수행
predictions = sgb_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_sgb'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_sgb'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_sgb
7109,7109,2024-07-19 13:29:00,162.75,162.80,162.59,162.63,1289.717,809,0.214352,0.235240,...,False,163.590931,161.769140,False,163.766759,161.623241,False,3.818484,-0.215212,1
2880,2880,2024-07-16 15:00:00,155.91,155.93,155.69,155.69,1983.029,900,0.219039,0.222679,...,True,156.862755,154.977555,True,156.426560,154.714806,False,3.796005,-0.057807,1
2881,2881,2024-07-16 15:01:00,155.70,155.78,155.60,155.71,1419.077,901,0.211231,0.218411,...,True,156.736100,154.977555,True,156.426560,154.633844,False,3.782673,-0.070644,1
7106,7106,2024-07-19 13:26:00,162.82,162.85,162.69,162.70,1820.981,806,0.233500,0.250055,...,False,163.704624,161.835376,False,163.937498,161.602502,False,3.773817,-0.258144,1
7107,7107,2024-07-19 13:27:00,162.69,162.78,162.53,162.72,2155.233,807,0.236800,0.250049,...,False,163.590931,161.719069,False,163.838998,161.471002,False,3.761062,-0.270403,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,8522,2024-07-20 13:02:00,168.56,168.73,168.56,168.73,846.118,782,0.132566,0.128740,...,True,168.816198,168.145976,False,169.307830,168.249485,True,0.000000,-0.865288,0
3519,3519,2024-07-17 01:39:00,163.03,163.15,163.03,163.15,619.675,99,0.219354,0.251539,...,True,164.098585,162.673690,True,164.186768,162.223585,True,0.000000,-1.121667,1
1075,1075,2024-07-15 08:55:00,152.72,153.06,152.72,153.06,1988.077,535,0.266985,0.238983,...,False,153.399907,152.140127,False,153.993659,151.555073,False,0.000000,-0.672939,1
7594,7594,2024-07-19 21:34:00,169.14,169.29,169.14,169.29,1151.051,1294,0.174188,0.174318,...,False,169.778412,168.489909,False,170.016173,168.344062,False,0.000000,-0.886054,0


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test['prediction_sgb'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test['prediction_sgb'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 13.435944913783127
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 25.47158893646569


######################

**Gradient Boosted Decision Trees (GBDT)**
######################

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Gradient Boosted Decision Trees (GBDT) 모델 생성 및 학습
gbdt_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
gbdt_model.fit(X_train, y_train)

# 예측 수행
y_pred_prob = gbdt_model.predict_proba(X_test)[:, 1]  # 클래스 1에 대한 확률
y_pred = (y_pred_prob > 0.5).astype(int)  # 임계값 0.5를 기준으로 0 또는 1로 변환

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Accuracy: 0.6843287441018681
Precision: 0.6498843809123397
Recall: 0.38947213706798806
F1 Score: 0.4870549312047054


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_gbdt_model.joblib'
joblib.dump(gbdt_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_gbdt_model.joblib


In [ ]:
# 예측 수행
predictions = gbdt_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_gbdt'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_gbdt'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_sgb,prediction_gbdt
7109,7109,2024-07-19 13:29:00,162.75,162.80,162.59,162.63,1289.717,809,0.214352,0.235240,...,163.590931,161.769140,False,163.766759,161.623241,False,3.818484,-0.215212,1,1
2880,2880,2024-07-16 15:00:00,155.91,155.93,155.69,155.69,1983.029,900,0.219039,0.222679,...,156.862755,154.977555,True,156.426560,154.714806,False,3.796005,-0.057807,1,1
2881,2881,2024-07-16 15:01:00,155.70,155.78,155.60,155.71,1419.077,901,0.211231,0.218411,...,156.736100,154.977555,True,156.426560,154.633844,False,3.782673,-0.070644,1,1
7106,7106,2024-07-19 13:26:00,162.82,162.85,162.69,162.70,1820.981,806,0.233500,0.250055,...,163.704624,161.835376,False,163.937498,161.602502,False,3.773817,-0.258144,1,1
7107,7107,2024-07-19 13:27:00,162.69,162.78,162.53,162.72,2155.233,807,0.236800,0.250049,...,163.590931,161.719069,False,163.838998,161.471002,False,3.761062,-0.270403,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,8522,2024-07-20 13:02:00,168.56,168.73,168.56,168.73,846.118,782,0.132566,0.128740,...,168.816198,168.145976,False,169.307830,168.249485,True,0.000000,-0.865288,0,0
3519,3519,2024-07-17 01:39:00,163.03,163.15,163.03,163.15,619.675,99,0.219354,0.251539,...,164.098585,162.673690,True,164.186768,162.223585,True,0.000000,-1.121667,1,1
1075,1075,2024-07-15 08:55:00,152.72,153.06,152.72,153.06,1988.077,535,0.266985,0.238983,...,153.399907,152.140127,False,153.993659,151.555073,False,0.000000,-0.672939,1,1
7594,7594,2024-07-19 21:34:00,169.14,169.29,169.14,169.29,1151.051,1294,0.174188,0.174318,...,169.778412,168.489909,False,170.016173,168.344062,False,0.000000,-0.886054,0,0


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test['prediction_gbdt'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test['prediction_gbdt'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 13.459090383057518
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 24.04814257609073


######################

**DART (Dropouts meet Multiple Additive Regression Trees)**
######################

In [ ]:
import lightgbm as lgb

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# LightGBM 데이터셋 생성
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# LightGBM 하이퍼파라미터 설정 (DART 사용)
params = {
    'boosting_type': 'dart',  # DART 사용
    'objective': 'binary',
    'metric': ['binary_logloss', 'auc'],
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'drop_rate': 0.1,  # Dropout rate
    'max_drop': 10,  # 최대 drop 수
    'skip_drop': 0.5,  # Dropout을 스킵할 확률
    'verbose': -1,
    'random_state': 42
}

# 모델 학습 (조기 종료 없이)
lgb_dart = lgb.train(
    params,
    lgb_train,
    num_boost_round=100,
    valid_sets=lgb_eval
)

# 예측 수행
y_pred_prob = lgb_dart.predict(X_test, num_iteration=lgb_dart.best_iteration)
y_pred = (y_pred_prob > 0.5).astype(int)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

X_train shape: (495071, 62)
X_test shape: (123768, 62)
y_train shape: (495071,)
y_test shape: (123768,)
Accuracy: 0.6913337857927736
Precision: 0.6528070573735932
Recall: 0.42262629656070216
F1 Score: 0.5130832664194038


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_lgb_dart.joblib'
joblib.dump(lgb_dart, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_lgb_dart.joblib


In [ ]:
# 예측 수행
predictions_prob = lgb_dart.predict(data_test_predict_scaled)

# 확률을 0 또는 1로 변환 (기본 임계값 0.5)
predictions = (predictions_prob > 0.5).astype(int)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_lgb_dart'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_lgb_dart'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,Unnamed: 0,open_time,open,high,low,close,volume,time,atr_5,atr_10,...,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min,prediction_sgb,prediction_gbdt,prediction_lgb_dart
7109,7109,2024-07-19 13:29:00,162.75,162.80,162.59,162.63,1289.717,809,0.214352,0.235240,...,161.769140,False,163.766759,161.623241,False,3.818484,-0.215212,1,1,0
2880,2880,2024-07-16 15:00:00,155.91,155.93,155.69,155.69,1983.029,900,0.219039,0.222679,...,154.977555,True,156.426560,154.714806,False,3.796005,-0.057807,1,1,1
2881,2881,2024-07-16 15:01:00,155.70,155.78,155.60,155.71,1419.077,901,0.211231,0.218411,...,154.977555,True,156.426560,154.633844,False,3.782673,-0.070644,1,1,1
7106,7106,2024-07-19 13:26:00,162.82,162.85,162.69,162.70,1820.981,806,0.233500,0.250055,...,161.835376,False,163.937498,161.602502,False,3.773817,-0.258144,1,1,0
7107,7107,2024-07-19 13:27:00,162.69,162.78,162.53,162.72,2155.233,807,0.236800,0.250049,...,161.719069,False,163.838998,161.471002,False,3.761062,-0.270403,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8522,8522,2024-07-20 13:02:00,168.56,168.73,168.56,168.73,846.118,782,0.132566,0.128740,...,168.145976,False,169.307830,168.249485,True,0.000000,-0.865288,0,0,0
3519,3519,2024-07-17 01:39:00,163.03,163.15,163.03,163.15,619.675,99,0.219354,0.251539,...,162.673690,True,164.186768,162.223585,True,0.000000,-1.121667,1,1,0
1075,1075,2024-07-15 08:55:00,152.72,153.06,152.72,153.06,1988.077,535,0.266985,0.238983,...,152.140127,False,153.993659,151.555073,False,0.000000,-0.672939,1,1,1
7594,7594,2024-07-19 21:34:00,169.14,169.29,169.14,169.29,1151.051,1294,0.174188,0.174318,...,168.489909,False,170.016173,168.344062,False,0.000000,-0.886054,0,0,0


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test['prediction_lgb_dart'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test['prediction_lgb_dart'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 13.192917486402036
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 28.00601782201134


######################

**TabNet**
######################

In [ ]:
pip install pytorch-tabnet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x8

In [ ]:
pip install pytorch-tabnet

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# GPU 설정 (가능한 경우)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import optuna

# Optuna를 사용한 하이퍼파라미터 튜닝
def objective(trial):
    param = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_a': trial.suggest_int('n_a', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'lambda_sparse': trial.suggest_float('lambda_sparse', 0.0001, 0.01),
        'optimizer_fn': torch.optim.Adam,
        'optimizer_params': dict(lr=trial.suggest_float('lr', 1e-5, 1e-1, log=True)),
        'mask_type': 'entmax',
        'scheduler_params': {"step_size":trial.suggest_int('step_size', 10, 50), "gamma":trial.suggest_float('scheduler_gamma', 0.5, 0.95)},
        'scheduler_fn': torch.optim.lr_scheduler.StepLR
    }

    clf = TabNetClassifier(**param)

    clf.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_name=['valid'],
        eval_metric=['accuracy'],
        max_epochs=200,
        patience=20,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=1,
        drop_last=False
    )

    preds = clf.predict(X_test)
    accuracy = accuracy_score(y_test, preds)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2024-07-24 01:00:48,844] A new study created in memory with name: no-name-4eb107a2-03e4-43ea-a07a-2936d25dae2d
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 0  | loss: 0.87701 | valid_accuracy: 0.52306 |  0:01:16s


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 1  | loss: 0.83499 | valid_accuracy: 0.54436 |  0:02:01s


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

epoch 198| loss: 0.63515 | valid_accuracy: 0.65994 |  2:33:43s


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 199| loss: 0.63551 | valid_accuracy: 0.6603  |  2:34:30s
Stop training because you reached max_epochs = 200 with best_epoch = 199 and best_valid_accuracy = 0.6603


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-24 03:37:17,269] Trial 0 finished with value: 0.6602999159718183 and parameters: {'n_d': 8, 'n_a': 28, 'n_steps': 4, 'gamma': 1.1024941103137174, 'n_independent': 1, 'n_shared': 1, 'lambda_sparse': 0.008387970291082432, 'lr': 1.675296346198297e-05, 'step_size': 27, 'scheduler_gamma': 0.7781037965643942}. Best is trial 0 with value: 0.6602999159718183.


Number of finished trials: 1
Best trial: {'n_d': 8, 'n_a': 28, 'n_steps': 4, 'gamma': 1.1024941103137174, 'n_independent': 1, 'n_shared': 1, 'lambda_sparse': 0.008387970291082432, 'lr': 1.675296346198297e-05, 'step_size': 27, 'scheduler_gamma': 0.7781037965643942}


In [ ]:
# 최적 하이퍼파라미터 출력
best_params = study.best_trial.params
print("Best hyperparameters found:")
for key, value in best_params.items():
    print(f"{key}: {value}")

Best hyperparameters found:
n_d: 8
n_a: 28
n_steps: 4
gamma: 1.1024941103137174
n_independent: 1
n_shared: 1
lambda_sparse: 0.008387970291082432
lr: 1.675296346198297e-05
step_size: 27
scheduler_gamma: 0.7781037965643942


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_trial.params
best_clf = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    n_independent=best_params['n_independent'],
    n_shared=best_params['n_shared'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['lr']),
    mask_type='entmax',
    scheduler_params={"step_size":best_params['step_size'], "gamma":best_params['scheduler_gamma']},
    scheduler_fn=torch.optim.lr_scheduler.StepLR
)

best_clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_name=['valid'],
    eval_metric=['accuracy'],
    max_epochs=200,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False
)

# 예측 수행
y_pred = best_clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 0  | loss: 0.87701 | valid_accuracy: 0.52306 |  0:00:51s


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 1  | loss: 0.83499 | valid_accuracy: 0.54436 |  0:01:43s


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

epoch 198| loss: 0.63515 | valid_accuracy: 0.65994 |  2:38:11s


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 199| loss: 0.63551 | valid_accuracy: 0.6603  |  2:38:57s
Stop training because you reached max_epochs = 200 with best_epoch = 199 and best_valid_accuracy = 0.6603


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Accuracy: 0.6602999159718183
Precision: 0.6166513416367132
Recall: 0.30978877083945744
F1 Score: 0.41239937388193204


In [ ]:
#임의 파라미터

from pytorch_tabnet.tab_model import TabNetClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# TabNet 모델 생성 및 학습
tabnet_model = TabNetClassifier()
tabnet_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# 예측 수행
y_pred = tabnet_model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.62625 | train_accuracy: 0.65877 | valid_accuracy: 0.65969 |  0:01:39s
epoch 1  | loss: 0.61104 | train_accuracy: 0.66637 | valid_accuracy: 0.66632 |  0:03:16s
epoch 2  | loss: 0.60831 | train_accuracy: 0.67054 | valid_accuracy: 0.67153 |  0:04:53s
epoch 3  | loss: 0.60673 | train_accuracy: 0.67343 | valid_accuracy: 0.67443 |  0:06:33s
epoch 4  | loss: 0.60382 | train_accuracy: 0.67669 | valid_accuracy: 0.67659 |  0:08:05s
epoch 5  | loss: 0.60328 | train_accuracy: 0.6744  | valid_accuracy: 0.67561 |  0:09:40s
epoch 6  | loss: 0.60258 | train_accuracy: 0.6785  | valid_accuracy: 0.67811 |  0:11:15s
epoch 7  | loss: 0.60123 | train_accuracy: 0.67926 | valid_accuracy: 0.67942 |  0:12:50s
epoch 8  | loss: 0.59915 | train_accuracy: 0.68008 | valid_accuracy: 0.67989 |  0:14:23s
epoch 9  | loss: 0.59765 | train_accuracy: 0.67314 | valid_accuracy: 0.67262 |  0:15:55s
epoch 10 | loss: 0.5966  | train_accuracy: 0.68249 | valid_accuracy: 0.68288 |  0:17:30s
epoch 11 | loss: 0.59

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Accuracy: 0.72148922589663
Precision: 0.6869741396291189
Recall: 0.5111599664991625
F1 Score: 0.5861675690601102


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_Tabnet.joblib'
joblib.dump(tabnet_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_Tabnet.joblib


In [ ]:
#2nd Account
# 모델 저장
model_path = '/content/drive/MyDrive/SO60_SMALL_tabnet_best.joblib'
joblib.dump(best_clf, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/SO60_SMALL_tabnet_model.joblib


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
# 모델 로딩
model_path_load = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_Tabnet.joblib'

tabnet_model = joblib.load(model_path_load)

print("Model loaded successfully")

Model loaded successfully


In [ ]:
# 예측 수행
predictions = tabnet_model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_tabnet_best'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_tabnet_best'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200,prediction_tabnet_best
1047753,2022-12-29 22:05:00,8.36,8.37,8.34,8.35,10561.71,1325,0.033625,0.034524,0.035585,...,8102.205,8.3880,8117.9660,8.3598,9636.0786,8.3398,21511.3793,8.61855,19199.22915,1
1047754,2022-12-29 22:06:00,8.34,8.36,8.34,8.35,1605.04,1326,0.030900,0.033071,0.034472,...,8048.333,8.3865,7848.2100,8.3602,9451.3368,8.3372,21419.9359,8.61425,19202.05880,1
1047749,2022-12-29 22:01:00,8.37,8.38,8.35,8.36,4238.04,1321,0.033849,0.034673,0.035973,...,7852.626,8.3955,8324.4860,8.3600,9897.3732,8.3482,22011.9198,8.63575,19156.98495,1
1047757,2022-12-29 22:09:00,8.37,8.38,8.36,8.36,1968.37,1329,0.025581,0.029529,0.031587,...,5951.699,8.3850,7058.8500,8.3630,9205.1186,8.3313,21116.8933,8.60175,19160.44735,1
1047735,2022-12-29 21:47:00,8.39,8.39,8.35,8.36,5033.92,1307,0.038514,0.040075,0.041471,...,8998.236,8.3555,10514.4910,8.3386,12683.3338,8.3861,23130.2088,8.69380,19024.55800,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048311,2022-12-30 07:23:00,9.26,9.27,9.23,9.25,22499.80,443,0.037005,0.034732,0.033375,...,12240.170,9.3210,8187.4865,9.3654,5662.3116,9.3803,10192.0166,9.47650,8251.54360,1
1047584,2022-12-29 19:16:00,9.09,9.09,9.04,9.07,27911.10,1156,0.034225,0.031308,0.030527,...,10691.936,9.1175,9358.6905,9.1732,8213.5380,9.2520,6533.1999,9.31975,4811.66080,1
1047633,2022-12-29 20:05:00,8.80,8.80,8.77,8.78,9039.50,1205,0.039851,0.041652,0.042575,...,8406.316,8.7850,14018.2790,8.8462,21907.8132,9.0114,14842.2311,9.19035,9394.87475,1
1047646,2022-12-29 20:18:00,8.70,8.70,8.67,8.68,7418.21,1218,0.039387,0.042609,0.043413,...,10982.462,8.7290,10722.8850,8.7854,14649.2924,8.9417,15533.1071,9.14345,9914.12480,1


In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.2) & (data_test['prediction_tabnet_best'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.2) & (data_test['prediction_tabnet_best'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 13.20609589184979
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 49.88761285172491


######################

**AutoGluon**
######################

In [15]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB

In [ ]:
from autogluon.tabular import TabularPredictor

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 학습 데이터와 검증 데이터를 DataFrame으로 변환
train_data = pd.DataFrame(X_train, columns=X.columns)
train_data['target'] = y_train.values

test_data = pd.DataFrame(X_test, columns=X.columns)
test_data['target'] = y_test.values

# AutoGluon 모델 학습
predictor = TabularPredictor(label='target', eval_metric='accuracy').fit(train_data)

# 검증 데이터에 대한 예측 수행
y_pred = predictor.predict(test_data)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

No path specified. Models will be saved in: "AutogluonModels/ag-20240725_091607"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       6.06 GB / 12.67 GB (47.8%)
Disk Space Avail:   71.19 GB / 107.72 GB (66.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fa

[1000]	valid_set's binary_error: 0.256918
[2000]	valid_set's binary_error: 0.223995
[3000]	valid_set's binary_error: 0.196324
[4000]	valid_set's binary_error: 0.172288
[5000]	valid_set's binary_error: 0.159766
[6000]	valid_set's binary_error: 0.146839
[7000]	valid_set's binary_error: 0.131691
[8000]	valid_set's binary_error: 0.119572
[9000]	valid_set's binary_error: 0.109271
[10000]	valid_set's binary_error: 0.102


	0.8988	 = Validation score   (accuracy)
	1994.15s	 = Training   runtime
	4.39s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.209251
[2000]	valid_set's binary_error: 0.144819
[3000]	valid_set's binary_error: 0.115734
[4000]	valid_set's binary_error: 0.100586
[5000]	valid_set's binary_error: 0.0874571
[6000]	valid_set's binary_error: 0.0759443
[7000]	valid_set's binary_error: 0.0700869
[8000]	valid_set's binary_error: 0.0630176
[9000]	valid_set's binary_error: 0.05918
[10000]	valid_set's binary_error: 0.0557463


	0.9445	 = Validation score   (accuracy)
	2114.58s	 = Training   runtime
	5.84s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9422	 = Validation score   (accuracy)
	3356.33s	 = Training   runtime
	0.76s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9529	 = Validation score   (accuracy)
	4314.37s	 = Training   runtime
	0.63s	 = Validation runtime
Fitting model: CatBoost ...
	0.6728	 = Validation score   (accuracy)
	23.99s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8802	 = Validation score   (accuracy)
	552.57s	 = Training   runtime
	1.12s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8818	 = Validation score   (accuracy)
	532.86s	 = Training   runtime
	0.55s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	To avoid this warning, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.10 to avoid the warning)
		To set the same value for all mo

[1000]	valid_set's binary_error: 0.112503
[2000]	valid_set's binary_error: 0.0725106
[3000]	valid_set's binary_error: 0.0616037
[4000]	valid_set's binary_error: 0.0519087
[5000]	valid_set's binary_error: 0.0446374
[6000]	valid_set's binary_error: 0.0410018
[7000]	valid_set's binary_error: 0.0399919
[8000]	valid_set's binary_error: 0.0371642
[9000]	valid_set's binary_error: 0.0359523
[10000]	valid_set's binary_error: 0.0345385


	0.9659	 = Validation score   (accuracy)
	3005.01s	 = Training   runtime
	8.87s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBMLarge': 0.778, 'XGBoost': 0.222}
	0.9663	 = Validation score   (accuracy)
	0.48s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 22607.19s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 239.8 rows/s (4951 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240725_091607")


Accuracy: 0.9654032916158327
Precision: 0.9616283020470568
Recall: 0.948178391959799
F1 Score: 0.9548559861679248


In [ ]:
# 최적의 모델 정보 출력
leaderboard = predictor.leaderboard(test_data, silent=True)
print("Leaderboard:")
print(leaderboard)

best_model = predictor.get_model_best()
print(f"Best Model: {best_model}")

best_model_params = predictor.info()['model_info'][best_model]['hyperparameters']
print("Best Model Hyperparameters:")
print(best_model_params)

KeyboardInterrupt: 

In [ ]:
# 특정 경로에서 모델 로드
save_path = "/content/drive/MyDrive/SO60_SMALL_AutoGluon"
predictor = TabularPredictor.load(save_path)

In [ ]:
# 예측 데이터를 DataFrame으로 변환
data_test_predict_df = pd.DataFrame(data_test_predict_scaled, columns=data_test_predict.columns)

# 예측 수행
predictions = predictor.predict(data_test_predict_df)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_AutoGluon'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_AutoGluon'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test['prediction_AutoGluon'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test['prediction_AutoGluon'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 31.678029342042112
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 20.113470547266736


######################

**모델분리**
######################

In [ ]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
from autogluon.tabular import TabularPredictor

# 모델 불러오기
save_path = '/content/drive/MyDrive/Data/AutogluonModels/ag-20240725_091607'
predictor = TabularPredictor.load(save_path)

In [30]:
#2nd Account
from autogluon.tabular import TabularPredictor

# 모델 불러오기
save_path = '/content/drive/MyDrive/AutogluonModels/ag-20240723_030958'
predictor = TabularPredictor.load(save_path)

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

This means that the predictor was fit in an AutoGluon version `<=0.3.1`.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AutogluonModels/ag-20240723_030958/predictor.pkl'

In [9]:
#"WeightedEnsemble_L2", "LightGBMLarge", "RandomForestEntr", "XGBoost", "RandomForestGini"
#"LightGBM", "ExtraTreesEntr", "ExtraTreesGini", "CatBoost", "LightGBMXT"
#"KNeighborsDist", "KNeighborsUnif "

select = "CatBoost"
predictor_sep = predictor._trainer.load_model(select)

#WeightedEnsemble_L2(autogluon) => , 실제확률(S) => , 파라미터 =>

In [89]:
test_data = pd.DataFrame(X_test, columns=X.columns)

# 검증 데이터에 대한 예측 수행
y_pred = predictor_sep.predict(test_data)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7248745647132966
Precision: 0.6749540535021442
Recall: 0.553643216080402
F1 Score: 0.6083095610564093


In [90]:
#하이퍼파라미터 파악
select_model_params = predictor.info()['model_info'][select]['hyperparameters']
print("Best Model Hyperparameters:")
print(select_model_params)

Best Model Hyperparameters:
{'weights': 'uniform'}


In [ ]:
# 모델 저장
model_path = f'/content/drive/MyDrive/Data/SOL60_automl_{select}.joblib'
joblib.dump(predictor_sep, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SOL60_automl_WeightedEnsemble_L2.joblib


In [ ]:
#2nd Account
# 모델 저장
model_path = f'/content/drive/MyDrive/SOL60_automl_{select}.joblib'
joblib.dump(predictor_sep, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/SOL60_automl_LightGBMLarge.joblib


In [10]:
# 예측 데이터를 DataFrame으로 변환
data_test_predict_df = pd.DataFrame(data_test_predict_scaled, columns=data_test_predict.columns)

# 예측 수행
predictions = predictor_sep.predict(data_test_predict_df)

# 예측 결과를 데이터프레임에 동적으로 추가
column_name = f'prediction_AutoGluon_{select}'
data_test[column_name] = predictions

In [11]:
# 결측치가 없는 행만 선택하여 새로운 데이터프레임 생성
data_test_non_nan = data_test.dropna(subset=[column_name])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,volume_ma_10,price_ma_20,volume_ma_20,price_ma_50,volume_ma_50,price_ma_100,volume_ma_100,price_ma_200,volume_ma_200,prediction_AutoGluon_CatBoost
1047753,2022-12-29 22:05:00,8.36,8.37,8.34,8.35,10561.71,1325,0.033625,0.034524,0.035585,...,8102.205,8.3880,8117.9660,8.3598,9636.0786,8.3398,21511.3793,8.61855,19199.22915,1
1047754,2022-12-29 22:06:00,8.34,8.36,8.34,8.35,1605.04,1326,0.030900,0.033071,0.034472,...,8048.333,8.3865,7848.2100,8.3602,9451.3368,8.3372,21419.9359,8.61425,19202.05880,1
1047749,2022-12-29 22:01:00,8.37,8.38,8.35,8.36,4238.04,1321,0.033849,0.034673,0.035973,...,7852.626,8.3955,8324.4860,8.3600,9897.3732,8.3482,22011.9198,8.63575,19156.98495,1
1047757,2022-12-29 22:09:00,8.37,8.38,8.36,8.36,1968.37,1329,0.025581,0.029529,0.031587,...,5951.699,8.3850,7058.8500,8.3630,9205.1186,8.3313,21116.8933,8.60175,19160.44735,1
1047735,2022-12-29 21:47:00,8.39,8.39,8.35,8.36,5033.92,1307,0.038514,0.040075,0.041471,...,8998.236,8.3555,10514.4910,8.3386,12683.3338,8.3861,23130.2088,8.69380,19024.55800,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048311,2022-12-30 07:23:00,9.26,9.27,9.23,9.25,22499.80,443,0.037005,0.034732,0.033375,...,12240.170,9.3210,8187.4865,9.3654,5662.3116,9.3803,10192.0166,9.47650,8251.54360,1
1047584,2022-12-29 19:16:00,9.09,9.09,9.04,9.07,27911.10,1156,0.034225,0.031308,0.030527,...,10691.936,9.1175,9358.6905,9.1732,8213.5380,9.2520,6533.1999,9.31975,4811.66080,1
1047633,2022-12-29 20:05:00,8.80,8.80,8.77,8.78,9039.50,1205,0.039851,0.041652,0.042575,...,8406.316,8.7850,14018.2790,8.8462,21907.8132,9.0114,14842.2311,9.19035,9394.87475,1
1047646,2022-12-29 20:18:00,8.70,8.70,8.67,8.68,7418.21,1218,0.039387,0.042609,0.043413,...,10982.462,8.7290,10722.8850,8.7854,14649.2924,8.9417,15533.1071,9.14345,9914.12480,1


In [12]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test[column_name] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test[column_name] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: 11.865529706242132
max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: 58.4112610145061


######################

**AutoGluon(시계열)**
######################

In [ ]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from autogluon.tabular import TabularPredictor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 학습 데이터 준비
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# AutoGluon 모델 학습
predictor = TabularPredictor(label='target', eval_metric='accuracy').fit(
    train_data=train_data,
    presets='best_quality',
    hyperparameters={
        'GBM': {'extra_trees': True, 'num_boost_round': 1000}  # GBMLarge의 하이퍼파라미터 설정
    }
)

# 모델 성능 평가
y_pred = predictor.predict(X_test)

# 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 평가 지표 출력
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# 모델 성능 상세 평가
leaderboard = predictor.leaderboard(test_data, extra_metrics=['accuracy', 'precision', 'recall', 'f1'])
print(leaderboard)

No path specified. Models will be saved in: "AutogluonModels/ag-20240724_122758"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          96
Memory Avail:       315.67 GB / 334.56 GB (94.4%)
Disk Space Avail:   202.62 GB / 225.33 GB (89.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon wil

Accuracy: 0.8339428934625349
Precision: 0.7519364484634506
Recall: 0.46572576146379563
F1 Score: 0.5751942898087417
                 model  score_test  accuracy  precision    recall        f1  \
0      LightGBM_BAG_L2    0.833943  0.833943   0.751936  0.465726  0.575194   
1  WeightedEnsemble_L3    0.833943  0.833943   0.751936  0.465726  0.575194   
2      LightGBM_BAG_L1    0.798393  0.798393   0.697276  0.291264  0.410892   
3  WeightedEnsemble_L2    0.798393  0.798393   0.697276  0.291264  0.410892   

   score_val eval_metric  pred_time_test  pred_time_val    fit_time  \
0   0.828778    accuracy       20.490596      39.077289  228.714161   
1   0.828778    accuracy       20.495692      39.360034  233.259134   
2   0.798991    accuracy        9.465401      18.212466   94.075903   
3   0.798991    accuracy        9.470607      18.453958   94.221927   

   pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  \
0                11.025195               20.864823         

In [ ]:
select = "LightGBM_BAG_L2"

#하이퍼파라미터 파악
select_model_params = predictor.info()['model_info'][select]['hyperparameters']
print("Best Model Hyperparameters:")
print(select_model_params)

AttributeError: 'StackerEnsembleModel' object has no attribute 'info'

In [ ]:
predictor_sep = predictor._trainer.load_model(select)

# 모델 저장
model_path = f'/content/drive/MyDrive/Data/SOL60_automl_{select}.joblib'
joblib.dump(predictor_sep, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SOL60_automl_LightGBM_BAG_L2.joblib


In [ ]:
from autogluon.tabular import TabularPredictor

# 모델 불러오기
save_path = '/content/AutogluonModels/ag-20240724_122758'
predictor = TabularPredictor.load(save_path)

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
select = "LightGBM_BAG_L2"
predictor_sep = predictor._trainer.load_model(select)

In [ ]:
# 모델 불러오기
save_path = '/content/drive/MyDrive/Data/SOL60_automl_LightGBM_BAG_L2.joblib'
predictor_sep = joblib.load(save_path)

In [ ]:
# 예측 데이터를 DataFrame으로 변환
data_test_predict_df = pd.DataFrame(data_test_predict_scaled, columns=data_test.columns)

# 예측 수행
predictions = predictor_sep.predict(data_test_predict_df)

# 예측 결과를 데이터프레임에 동적으로 추가
column_name = f'prediction_AutoGluon_{select}'
data_test[column_name] = predictions

FileNotFoundError: [Errno 2] No such file or directory: '/content/AutogluonModels/ag-20240724_122758/models/LightGBM_BAG_L2/../../../../LightGBM_BAG_L1/model.pkl'

In [ ]:
# 결측치가 없는 행만 선택하여 새로운 데이터프레임 생성
data_test_non_nan = data_test.dropna(subset=[column_name])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test[column_name] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test[column_name] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

######################

**Transformer(시계열)**
######################

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
# 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# TransformerEncoder 모델 정의
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_dim, nhead, num_layers, dim_feedforward, output_dim):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_dim, input_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src):
        src = self.embedding(src)
        output = self.transformer_encoder(src)
        output = self.fc(output[:, -1, :])  # Use the output from the last time step
        return output

# 모델 설정
input_dim = X_train.shape[2]
nhead = 2
num_layers = 2
dim_feedforward = 64
output_dim = 1

model = TransformerEncoderModel(input_dim, nhead, num_layers, dim_feedforward, output_dim)

# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        y_true.extend(y_batch.tolist())
        y_pred.extend(torch.sigmoid(output).squeeze().tolist())

# 이진 분류 결과를 위한 평가 지표 계산
y_pred = np.array(y_pred) > 0.5
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
#2nd Account
# 모델 저장 경로
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_Transformer.pth'

# 모델 상태 저장
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
#2nd Account
# 모델 저장 경로
model_path = '/content/drive/MyDrive/SOL60_SMALLL_INDICATOR3_Transformer.pth'

# 모델 상태 저장
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)

# 모델 정의 (로드할 때 필요)
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_dim, nhead, num_layers, dim_feedforward, output_dim):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_dim, input_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src):
        src = self.embedding(src)
        output = self.transformer_encoder(src)
        output = self.fc(output[:, -1, :])  # Use the output from the last time step
        return output

# 입력 차원 확인 및 설정
input_dim = X_test_seq.shape[2]
print(f"Input dimension: {input_dim}")

# 모델 설정 (로드할 때 필요)
nhead = 2
num_layers = 2
dim_feedforward = 64
output_dim = 1

model = TransformerEncoderModel(input_dim, nhead, num_layers, dim_feedforward, output_dim)

# 모델 로드
model_path = '/content/drive/MyDrive/SOL60_SMALLL_INDICATOR3_Transformer.pth'
model.load_state_dict(torch.load(model_path))
model.eval()
print(f"Model loaded from {model_path}")

In [ ]:
# sequence_length를 사용하여 X_test_seq의 shape를 맞춤
sequence_length = X_train.shape[1]
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).view(-1, sequence_length, input_dim)

# 예측 수행
with torch.no_grad():
    predictions = torch.sigmoid(model(X_test_tensor)).squeeze().numpy()

# 예측 결과를 이진 분류로 변환 (0 또는 1)
predictions = (predictions > 0.5).astype(int)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_Transformer'] = np.nan  # 예측 결과를 담을 열을 초기화
data_test.iloc[sequence_length - 1:sequence_length - 1 + len(predictions), data_test.columns.get_loc('prediction_Transformer')] = predictions

# 결과 확인
print(data_test[['open_time', 'prediction_Transformer']].head())

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_Transformer'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.1) & (data_test['prediction_Transformer'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.1) & (data_test['prediction_Transformer'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

######################

**TCN(시계열)**
######################

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# TCN 모델 정의
class TCNModel(nn.Module):
    def __init__(self, input_size, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        self.tcn = nn.Conv1d(input_size, num_channels, kernel_size, padding=kernel_size//2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels, 1)

    def forward(self, x):
        x = x.transpose(1, 2)  # (batch_size, seq_len, input_size) -> (batch_size, input_size, seq_len)
        y1 = self.tcn(x)
        y1 = self.relu(y1)
        y1 = self.dropout(y1)
        y1 = y1[:, :, -1]
        o = self.fc(y1)
        return o

# 모델 설정
input_size = X_train.shape[2]
num_channels = 64
model = TCNModel(input_size, num_channels)

# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        y_true.extend(y_batch.tolist())
        y_pred.extend(torch.sigmoid(output).squeeze().tolist())

# 이진 분류 결과를 위한 평가 지표 계산
y_pred = np.array(y_pred) > 0.5
accuracy = np.mean(y_pred == y_test_tensor.numpy())
precision = np.sum((y_pred == 1) & (y_test_tensor.numpy() == 1)) / np.sum(y_pred == 1)
recall = np.sum((y_pred == 1) & (y_test_tensor.numpy() == 1)) / np.sum(y_test_tensor.numpy() == 1)
f1 = 2 * (precision * recall) / (precision + recall)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# 모델 저장 경로
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_TCN.pth'

# 모델 상태 저장
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
#2nd Account
# 모델 저장 경로
model_path = '/content/drive/MyDrive/SOL60_SMALLL_INDICATOR3_TCN.pth'

# 모델 상태 저장
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
#모델 로드
class TCNModel(nn.Module):
    def __init__(self, input_channels, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        self.tcn = nn.Conv1d(input_channels, num_channels, kernel_size, padding=kernel_size//2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels, 1)

    def forward(self, x):
        x = x.transpose(1, 2)  # (batch_size, seq_len, input_channels) -> (batch_size, input_channels, seq_len)
        y1 = self.tcn(x)
        y1 = self.relu(y1)
        y1 = self.dropout(y1)
        y1 = y1[:, :, -1]
        o = self.fc(y1)
        return o

# 테스트 데이터 텐서로 변환
X_test_tensor = torch.tensor(X_seq, dtype=torch.float32)

# 입력 차원 확인 및 설정
input_channels = X_seq.shape[2]
print(f"Input channels: {input_channels}")

# 모델 설정
num_channels = 64
model = TCNModel(input_channels, num_channels)

# 모델 로드
model_path = '/kaggle/input/lstm_indiactor3/pytorch/tcn_transformer/1/SOL60_SMALLL_INDICATOR3_TCN.pth'
model.load_state_dict(torch.load(model_path))
model.train()  # 모델을 학습 모드로 설정
print(f"Model loaded from {model_path}")

In [ ]:
#추가 학습
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 조기 종료 설정
patience = 5
best_loss = float('inf')
patience_counter = 0

# 학습 및 검증 손실을 저장할 리스트
train_losses = []
val_losses = []

# 추가 학습
num_epochs = 100  # 최대 에포크 수
for epoch in range(num_epochs):
    # 학습 단계
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # 검증 단계
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            loss = criterion(output, y_batch)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(test_loader)
    val_losses.append(avg_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    # 조기 종료 조건 체크
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("조기 종료 조건 충족. 학습을 중지합니다.")
        break

# 학습 및 검증 손실 시각화
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 모델 평가
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        y_true.extend(y_batch.tolist())
        y_pred.extend(torch.sigmoid(output).squeeze().tolist())

# 이진 분류 결과를 위한 평가 지표 계산
y_pred = np.array(y_pred) > 0.5
accuracy = np.mean(y_pred == y_test_tensor.numpy())
precision = np.sum((y_pred == 1) & (y_test_tensor.numpy() == 1)) / np.sum(y_pred == 1)
recall = np.sum((y_pred == 1) & (y_test_tensor.numpy() == 1)) / np.sum(y_test_tensor.numpy() == 1)
f1 = 2 * (precision * recall) / (precision + recall)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# 추가 학습 모델 저장 경로
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_TCN_v2.pth'

# 모델 상태 저장
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
# 테스트 데이터 텐서로 변환
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)

# 입력 차원 확인 및 설정
input_channels = X_test_seq.shape[2]
print(f"Input channels: {input_channels}")

# 모델 설정
num_channels = 64
model = TCNModel(input_channels, num_channels)

# 모델 로드
model_path = '/kaggle/working/SOL60_SMALLL_INDICATOR3_TCN.pth'
model.load_state_dict(torch.load(model_path))
model.eval()
print(f"Model loaded from {model_path}")

# 예측 수행
with torch.no_grad():
    predictions = torch.sigmoid(model(X_test_tensor)).squeeze().numpy()

# 예측 결과를 이진 분류로 변환 (0 또는 1)
predictions = (predictions > 0.5).astype(int)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_TCN'] = np.nan  # 예측 결과를 담을 열을 초기화
data_test.iloc[sequence_length - 1:sequence_length - 1 + len(predictions), data_test.columns.get_loc('prediction_TCN')] = predictions

# 결과 확인
print(data_test[['open_time', 'prediction_TCN']].head())

In [ ]:
# sequence_length를 사용하여 X_test_seq의 shape를 맞춤
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)

# 예측 수행
model.eval()
with torch.no_grad():
    predictions = torch.sigmoid(model(X_test_tensor)).squeeze().numpy()

# 예측 결과를 이진 분류로 변환 (0 또는 1)
predictions = (predictions > 0.5).astype(int)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_Transformer'] = np.nan  # 예측 결과를 담을 열을 초기화
data_test.iloc[sequence_length - 1:sequence_length - 1 + len(predictions), data_test.columns.get_loc('prediction_Transformer')] = predictions

# 결과 확인
print(data_test[['open_time', 'prediction_Transformer']].head())

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_TCN'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.2) & (data_test['prediction_TCN'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.2) & (data_test['prediction_TCN'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

######################

**ARIMAX(시계열)**
######################

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# X_train과 y_train의 크기 확인
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# X_train과 X_test를 2차원 배열로 변환
X_train_2d = X_train.reshape(X_train.shape[0], -1)
X_test_2d = X_test.reshape(X_test.shape[0], -1)

# ARIMAX 모델 정의 및 학습
# ARIMAX 모델은 특정 파라미터 (p, d, q)를 설정해야 함
p, d, q = 5, 1, 0  # 이 값들은 데이터를 기반으로 튜닝되어야 함

model = SARIMAX(endog=y_train, exog=X_train_2d, order=(p, d, q))
arimax_model = model.fit(disp=False)

# 예측 수행
y_pred = arimax_model.predict(start=0, end=len(y_test)-1, exog=X_test_2d)

# 평가 지표 계산
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse:.4f}')
print(f'MAE: {mae:.4f}')
print(f'RMSE: {rmse:.4f}')

# 예측 결과 시각화
plt.plot(y_test, label='True')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
# 평가 지표 계산 (이진 분류)
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_ARIMAX.joblib'
joblib.dump(arimax_model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
#2nd Account
# 모델 저장
model_path = '/content/drive/MyDrive/SOL60_SMALLL_INDICATOR3_ARIMAX.joblib'
joblib.dump(arimax_model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
# 예측 수행
predictions = arimax_model.predict(start=len(data_train), end=len(data_train) + len(data_test_predict_scaled) - 1, exog=data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_ARIMAX'] = predictions

# 결과 확인
print(data_test[['open_time', 'prediction_ARIMAX']].head())

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_ARIMAX'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.2) & (data_test['prediction_ARIMAX'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.2) & (data_test['prediction_ARIMAX'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

######################

**Prophet(시계열/비시계열 데이터와 연결)**
######################

In [ ]:
pip install statsmodels

In [ ]:
from fbprophet import Prophet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Prophet을 사용하기 위해 'ds'와 'y' 컬럼을 생성합니다.
data['ds'] = pd.to_datetime(data['open_time'])  # 날짜/시간 열이 있는 경우
data['y'] = data['target']

# 학습 데이터와 검증 데이터 분리
train_data = data[['ds', 'y']].iloc[:-1000]  # 마지막 1000개를 검증 데이터로 사용
test_data = data[['ds', 'y']].iloc[-1000:]

# Prophet 모델 정의 및 학습
model = Prophet()
model.fit(train_data)

# 미래 데이터프레임 생성
future = model.make_future_dataframe(periods=1000, freq='T')  # 1000분 예측
forecast = model.predict(future)

# 예측 결과
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

# 평가 지표 계산 (이진 분류)
# 실제 데이터의 y와 예측 데이터의 yhat을 이진 분류로 변환
test_data = test_data.set_index('ds').join(forecast.set_index('ds')[['yhat']], how='left').reset_index()
test_data['y_pred'] = (test_data['yhat'] > 0.5).astype(int)

# 평가 지표 계산
accuracy = accuracy_score(test_data['y'], test_data['y_pred'])
precision = precision_score(test_data['y'], test_data['y_pred'])
recall = recall_score(test_data['y'], test_data['y_pred'])
f1 = f1_score(test_data['y'], test_data['y_pred'])

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# 예측 결과 시각화
import matplotlib.pyplot as plt
model.plot(forecast)
plt.show()

model.plot_components(forecast)
plt.show()

In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_Prophet.joblib'
joblib.dump(model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
#2nd Account
# 모델 저장
model_path = '/content/drive/MyDrive/SOL60_SMALLL_INDICATOR3_Prophet.joblib'
joblib.dump(model, model_path)

print(f"Model saved to {model_path}")

In [ ]:
# 예측 수행
predictions = model.predict(data_test_predict_scaled)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_Prophet'] = predictions

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_Prophet'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.2) & (data_test['prediction_Prophet'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.2) & (data_test['prediction_Prophet'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")

######################

**LSTM(시계열)**
######################

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 모델 설정
input_size = X_train.shape[2]
hidden_size = 50
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        y_true.extend(y_batch.tolist())
        y_pred.extend(torch.sigmoid(output).squeeze().tolist())

# 이진 분류 결과를 위한 평가 지표 계산
y_pred = np.array(y_pred) > 0.5
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# 모델 저장
model_path = '/content/drive/MyDrive/Data/SOL60_SMALLL_INDICATOR3_LSTM.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
#2nd Account
# 모델 저장
model_path = '/content/drive/MyDrive/SOL60_SMALLL_INDICATOR3_LSTM.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 모델 로드 경로
model_path = '/kaggle/input/lstm_indiactor3/keras/default/1/SOL60_SMALLL_INDICATOR3_LSTM.pth'

# 저장된 모델의 상태 사전 로드
state_dict = torch.load(model_path)

# 상태 사전의 키와 크기 확인
for key, value in state_dict.items():
    print(f"{key}: {value.shape}")

# input_dim과 hidden_dim 추출
lstm_weight_ih_l0_shape = state_dict['lstm.weight_ih_l0'].shape
input_dim = lstm_weight_ih_l0_shape[1]  # feature 개수
hidden_dim = lstm_weight_ih_l0_shape[0] // 4  # LSTM weight shape가 4 * hidden_dim 이므로 나누기 4

# 모델 설정
num_layers = 2
output_dim = 1

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)

# 모델 로드
model.load_state_dict(state_dict)
model.eval()
print(f"Model loaded from {model_path}")

In [ ]:
# 데이터 텐서로 변환
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)

# 예측 수행
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).cpu().numpy()

# 예측 결과를 이진 분류로 변환 (필요에 따라)
predictions_binary = (predictions > 0.5).astype(int)

# 예측 결과를 데이터프레임에 추가
data_test['prediction_LSTM'] = np.nan  # 예측 결과를 담을 열을 초기화
data_test.iloc[sequence_length - 1:sequence_length - 1 + len(predictions), data_test.columns.get_loc('prediction_LSTM')] = predictions_binary

# 결과 확인
print(data_test[['open_time', 'prediction_LSTM']].head())

In [ ]:
data_test_non_nan = data_test.dropna(subset=['prediction_LSTM'])

# max_return_60min 기준으로 내림차순 정렬
data_test_sorted = data_test_non_nan.sort_values(by='max_return_60min', ascending=False)

# 결과 출력
data_test_sorted

In [ ]:
# 'max_return_60min' 값이 1 이상이고 'prediction' 값이 0인 데이터의 개수
count_max_return_ge_1_prediction_0 = len(data_test[(data_test['max_return_60min'] >= 1.2) & (data_test['prediction_LSTM'] == 1)])

# 'max_return_60min' 값이 1 미만이고 'prediction' 값이 1인 데이터의 개수
count_max_return_lt_1_prediction_1 = len(data_test[(data_test['max_return_60min'] < 1.2) & (data_test['prediction_LSTM'] == 0)])

print(f"max_return_60min이 1 이상인데 prediction이 0인 데이터의 비율: {count_max_return_ge_1_prediction_0/len(data_test)*100}")
print(f"max_return_60min이 1 미만인데 prediction이 1인 데이터의 비율: {count_max_return_lt_1_prediction_1/len(data_test)*100}")